# Library


In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from datetime import datetime
import pyexasol
# import psycopg2

In [2]:
# 소수점 옵션
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

# folder

In [3]:
ods_fold = 'D:/data/big2/ODS'
df3_fold = 'D:/data/big2/BD3/df'

# server

In [4]:
# # insider db
# wd = pyexasol.connect(dsn='172.29.135.35/F99FAB2444F86051A9A467F6313FAAB48AF7C4760663430958E3B89A9DC53361:8563', user='sys', password='exasol', compression=True, schema='VSYSD')
# we = pyexasol.connect(dsn='172.29.135.35/F99FAB2444F86051A9A467F6313FAAB48AF7C4760663430958E3B89A9DC53361:8563', user='sys', password='exasol', compression=True, schema='VSYSE')

In [5]:
# exasol db
# ws = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYS')
wd = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYSD')
we = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYSE')

# Load
- 등록&제원 -> result 5등급

## 등록정보(STD_CEG_CAR_MIG) 5등급만

In [6]:
# 8.6s
car = wd.export_to_pandas("SELECT VIN, BSPL_STDG_CD, VHCL_ERSR_YN, MANP_MNG_NO, YRIDNW, VHCTY_CD, PURPS_CD2, FRST_REG_YMD, VHCL_FBCTN_YMD, VHCL_MNG_NO FROM STD_CEG_CAR_MIG WHERE EXHST_GAS_GRD_CD = 'A0505' OR EXHST_GAS_GRD_CD = 'A05T5';")

C:\Users\kbjung\AppData\Roaming\Python\Python39\site-packages\pyexasol\callback.py:42: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  return pandas.read_csv(pipe, skip_blank_lines=False, **kwargs)


In [7]:
car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2314337 entries, 0 to 2314336
Data columns (total 10 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   VIN             object
 1   BSPL_STDG_CD    int64 
 2   VHCL_ERSR_YN    object
 3   MANP_MNG_NO     object
 4   YRIDNW          int64 
 5   VHCTY_CD        object
 6   PURPS_CD2       object
 7   FRST_REG_YMD    object
 8   VHCL_FBCTN_YMD  object
 9   VHCL_MNG_NO     object
dtypes: int64(2), object(8)
memory usage: 176.6+ MB


In [8]:
car_ch_col = {
    'VIN':'차대번호', 
    'BSPL_STDG_CD':'법정동코드', 
    'VHCL_ERSR_YN':'차량말소YN',
    'MANP_MNG_NO':'제원관리번호',
    'YRIDNW':'차량연식', 
    'VHCTY_CD':'차종', 
    'PURPS_CD2':'용도', 
    'FRST_REG_YMD':'최초등록일자',
    'VHCL_FBCTN_YMD':'제작일자',
    'VHCL_MNG_NO':'차량관리번호'
}

In [9]:
carr = car.rename(columns=car_ch_col)

In [10]:
carr.columns

Index(['차대번호', '법정동코드', '차량말소YN', '제원관리번호', '차량연식', '차종', '용도', '최초등록일자',
       '제작일자', '차량관리번호'],
      dtype='object')

## 제원정보(STD_CEG_CAR_SRC_MIG)

In [11]:
# 3.8s
src = wd.export_to_pandas("SELECT MANP_MNG_NO, FUEL_CD, VHCTY_TY_CD2 FROM STD_CEG_CAR_SRC_MIG;")

In [12]:
src_ch_col = {
    'MANP_MNG_NO':'제원관리번호', 
    'FUEL_CD':'연료',
    'VHCTY_TY_CD2':'차종유형', 
}

In [13]:
srcr = src.rename(columns=src_ch_col)
srcr.columns

Index(['제원관리번호', '연료', '차종유형'], dtype='object')

## 저감장치 부착이력(STD_DLM_TB_ERP_ATT_HIS)

In [14]:
# 3.0s
att = wd.export_to_pandas("SELECT VIN, RDCDVC_SE_CD FROM STD_DLM_TB_ERP_ATT_HIS;")
att.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1015941 entries, 0 to 1015940
Data columns (total 2 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   VIN           1015941 non-null  object
 1   RDCDVC_SE_CD  1015941 non-null  object
dtypes: object(2)
memory usage: 15.5+ MB


In [15]:
att_ch_col = {
    'VIN':'차대번호', 
    'RDCDVC_SE_CD':'저감장치구분', 
}

In [16]:
attr = att.rename(columns=att_ch_col)
attr.columns

Index(['차대번호', '저감장치구분'], dtype='object')

## 법정동코드(STD_BJCD_INFO)

In [17]:
# 1.3s
code = wd.export_to_pandas("SELECT STDG_CD, STDG_CTPV_CD, STDG_SGG_CD, STDG_CTPV_NM, STDG_SGG_NM FROM STD_BJCD_INFO;")
code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47916 entries, 0 to 47915
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   STDG_CD       47916 non-null  int64 
 1   STDG_CTPV_CD  47916 non-null  int64 
 2   STDG_SGG_CD   47916 non-null  int64 
 3   STDG_CTPV_NM  47916 non-null  object
 4   STDG_SGG_NM   47892 non-null  object
dtypes: int64(3), object(2)
memory usage: 1.8+ MB


In [18]:
code.columns

Index(['STDG_CD', 'STDG_CTPV_CD', 'STDG_SGG_CD', 'STDG_CTPV_NM',
       'STDG_SGG_NM'],
      dtype='object')

In [19]:
code_ch_col = {
    'STDG_CD':'법정동코드', 
    'STDG_CTPV_NM':'시도', 
    'STDG_SGG_NM':'시군구', 
    'STDG_CTPV_CD':'시도코드', 
    'STDG_SGG_CD':'시군구코드',
}

In [20]:
coder = code.rename(columns=code_ch_col)
coder.columns

Index(['법정동코드', '시도코드', '시군구코드', '시도', '시군구'], dtype='object')

## 노후차 조기폐차 관리정보(수도권)(STD_DLM_TB_ERP_EARLY_ERASE_AEA)

In [21]:
# 2.4s
aea = wd.export_to_pandas("SELECT VIN, ELPDSRC_STTS_CD, ELPDSRC_LST_APRV_YN, ERSR_YMD FROM STD_DLM_TB_ERP_EARLY_ERASE_AEA;")
aea.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 811412 entries, 0 to 811411
Data columns (total 4 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   VIN                  811412 non-null  object 
 1   ELPDSRC_STTS_CD      811412 non-null  object 
 2   ELPDSRC_LST_APRV_YN  710812 non-null  object 
 3   ERSR_YMD             718610 non-null  float64
dtypes: float64(1), object(3)
memory usage: 24.8+ MB


In [22]:
aea.columns

Index(['VIN', 'ELPDSRC_STTS_CD', 'ELPDSRC_LST_APRV_YN', 'ERSR_YMD'], dtype='object')

In [23]:
aea_ch_col = {
    'VIN':'차대번호', 
    'ELPDSRC_STTS_CD':'조기폐차상태코드', 
    'ELPDSRC_LST_APRV_YN':'조기폐차최종승인YN', 
    'ERSR_YMD':'말소일자',
}

In [24]:
aear = aea.rename(columns=aea_ch_col)
aear.columns

Index(['차대번호', '조기폐차상태코드', '조기폐차최종승인YN', '말소일자'], dtype='object')

In [25]:
aear.head()

,차대번호,조기폐차상태코드,조기폐차최종승인YN,말소일자
0,KMZKL17BP1U441173,A32Y,Y,20130917.00
1,KNHUP7513WS711743,A32Y,Y,20100802.00
2,KPBEA3D81TP079170,A32Y,Y,20100806.00
3,KPBEA3D81SP063998,A32Y,Y,20100802.00
4,KPBEA3D81RP011984,A32Y,Y,20100802.00


## 노후차 조기폐차 관리정보(수도권외)(STD_DLM_TB_ERP_EARLY_ERASE_LGV)

In [26]:
# 1.8s
lgv = wd.export_to_pandas("SELECT VIN, ELPDSRC_STTS_CD, ELPDSRC_LST_APRV_YN, ERSR_YMD FROM STD_DLM_TB_ERP_EARLY_ERASE_LGV;")
lgv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 388958 entries, 0 to 388957
Data columns (total 4 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   VIN                  388958 non-null  object 
 1   ELPDSRC_STTS_CD      388958 non-null  object 
 2   ELPDSRC_LST_APRV_YN  176711 non-null  object 
 3   ERSR_YMD             190163 non-null  float64
dtypes: float64(1), object(3)
memory usage: 11.9+ MB


In [27]:
lgv.columns

Index(['VIN', 'ELPDSRC_STTS_CD', 'ELPDSRC_LST_APRV_YN', 'ERSR_YMD'], dtype='object')

In [28]:
lgv_ch_col = {
    'VIN':'차대번호', 
    'ELPDSRC_STTS_CD':'조기폐차상태코드', 
    'ELPDSRC_LST_APRV_YN':'조기폐차최종승인YN', 
    'ERSR_YMD':'말소일자', 
}

In [29]:
lgvr = lgv.rename(columns=lgv_ch_col)
lgvr.columns

Index(['차대번호', '조기폐차상태코드', '조기폐차최종승인YN', '말소일자'], dtype='object')

In [30]:
lgvr.head()

,차대번호,조기폐차상태코드,조기폐차최종승인YN,말소일자
0,KMHSD81VP4U668720,A32M,NaN,NaN
1,KPBEA2MC12P281515,A32M,NaN,NaN
2,KMHMH81VP2U154512,A32M,NaN,NaN
3,KMFWVH7HP6U722855,A32M,NaN,NaN
4,KN90J4MDD2GZ00019,A32C,NaN,NaN


## [❗] 비상시 및 계절제 단속발령(N_IS_ISSUE_DISCLOSURE)

In [31]:
# # 1m 27.5s
# isdis = pd.read_csv(os.path.join(ods_fold, 'ODS_N_IS_ISSUE_DISCLOSURE(col_sliced).csv'), low_memory=False)
# isdis_ch_col = {
#     'REGLT_NO':'적발번호', 
#     'GNFD_NO':'발령번호', 
#     'REG_SIDO_CD':'등록시도코드', 
#     'REG_SIGNGU_CD':'등록시군구코드', 
#     'VIN':'차대번호',
#     'REGLT_AREA_CD':'적발지역코드', 
# }
# isdisr = isdis.rename(columns=isdis_ch_col)
# isdisr.columns

In [32]:
# # 1.8s
# edb_id = 'vsysd'
# edb_database = 'edb'
# edb_port = 5444
# edb_url = '172.29.135.50'
# edb_pwd = 'vsyswynn'
# conn = psycopg2.connect(dbname=edb_database, user=edb_id, password=edb_pwd, host=edb_url, port=edb_port)
# cur = conn.cursor()
# sql = "select REGLT_NO, GNFD_NO, VIN, REG_SIDO_CD, REG_SIGNGU_CD, REGLT_AREA_CD from vsysd.n_is_issue_disclosure"
# cur.execute(sql)
# isdis = pd.DataFrame(cur.fetchall())

# isdis.columns = [desc[0].upper() for desc in cur.description]
# isdis_ch_col = {
#     'REGLT_NO':'적발번호', 
#     'GNFD_NO':'발령번호', 
#     'REG_SIDO_CD':'등록시도코드', 
#     'REG_SIGNGU_CD':'등록시군구코드', 
#     'VIN':'차대번호',
#     'REGLT_AREA_CD':'적발지역코드', 
# }
# isdisr = isdis.rename(columns=isdis_ch_col)
# isdisr.columns

In [33]:
isdis = wd.export_to_pandas("SELECT REGLT_NO, GNFD_NO, VIN, REG_SIDO_CD, REG_SIGNGU_CD, REGLT_AREA_CD FROM N_IS_ISSUE_DISCLOSURE;")
isdis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108436093 entries, 0 to 108436092
Data columns (total 6 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   REGLT_NO       int64 
 1   GNFD_NO        int64 
 2   VIN            object
 3   REG_SIDO_CD    int64 
 4   REG_SIGNGU_CD  int64 
 5   REGLT_AREA_CD  int64 
dtypes: int64(5), object(1)
memory usage: 4.8+ GB


In [34]:
isdis_ch_col = {
    'REGLT_NO':'적발번호', 
    'GNFD_NO':'발령번호', 
    'REG_SIDO_CD':'등록시도코드', 
    'REG_SIGNGU_CD':'등록시군구코드', 
    'VIN':'차대번호',
    'REGLT_AREA_CD':'적발지역코드', 
}
isdisr = isdis.rename(columns=isdis_ch_col)
isdisr.columns

Index(['적발번호', '발령번호', '차대번호', '등록시도코드', '등록시군구코드', '적발지역코드'], dtype='object')

## [❗] 운행제한 발령정보(N_IS_ISSUE)

In [35]:
# # 1s
# isis = pd.read_csv(os.path.join(ods_fold, 'ODS_N_IS_ISSUE.csv'), low_memory=False, usecols=['GNFD_NO', 'DNSTY_STDR_ID', 'TY_STDR_ID'])
# isis_ch_col = {
#     'GNFD_NO':'발령번호', 
#     'DNSTY_STDR_ID':'농도기준아이디', 
#     'TY_STDR_ID':'유형기준아이디', 
# }
# isisr = isis.rename(columns=isis_ch_col)
# isisr.columns

In [36]:
# sql = "select GNFD_NO, TY_STDR_ID, DNSTY_STDR_ID from vsysd.n_is_issue"
# cur.execute(sql)
# isis = pd.DataFrame(cur.fetchall())

# isis.columns = [desc[0].upper() for desc in cur.description]
# isis_ch_col = {
#     'GNFD_NO':'발령번호', 
#     'DNSTY_STDR_ID':'농도기준아이디', 
#     'TY_STDR_ID':'유형기준아이디', 
# }
# isisr = isis.rename(columns=isis_ch_col)
# isisr.columns

In [37]:
isis = wd.export_to_pandas("SELECT GNFD_NO, TY_STDR_ID, DNSTY_STDR_ID FROM N_IS_ISSUE;")
isis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2368 entries, 0 to 2367
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   GNFD_NO        2368 non-null   int64 
 1   TY_STDR_ID     2368 non-null   object
 2   DNSTY_STDR_ID  2368 non-null   object
dtypes: int64(1), object(2)
memory usage: 55.6+ KB


In [38]:
isis_ch_col = {
    'GNFD_NO':'발령번호', 
    'DNSTY_STDR_ID':'농도기준아이디', 
    'TY_STDR_ID':'유형기준아이디', 
}
isisr = isis.rename(columns=isis_ch_col)
isisr.columns

Index(['발령번호', '유형기준아이디', '농도기준아이디'], dtype='object')

In [39]:
# DNSTY_STDR_ID(농도기준아이디) : 실발령(C011), 모의발령(C012)
# TY_STDR_ID(유형기준아이디) : 비상시(T001), 계절제(T002)

## [❗] N_IS_PENALTY

In [40]:
# # 13.4s
# ispe = pd.read_csv(os.path.join(ods_fold, 'N_IS_PENALTY.csv'), low_memory=False, sep='|', usecols=['REGLT_NO', 'REGLT_DE'])
# ispe_ch_col = {
#     'REGLT_NO':'적발번호', 
#     'REGLT_DE':'단속일', 
# }
# isper = ispe.rename(columns=ispe_ch_col)
# isper.columns

In [41]:
# sql = "select REGLT_NO, REGLT_DE from vsysd.n_is_penalty"
# cur.execute(sql)
# ispe = pd.DataFrame(cur.fetchall())
# ispe.columns = [desc[0].upper() for desc in cur.description]
# ispe_ch_col = {
#     'REGLT_NO':'적발번호', 
#     'REGLT_DE':'단속일', 
# }
# isper = ispe.rename(columns=ispe_ch_col)
# isper.columns

In [42]:
ispe = wd.export_to_pandas("SELECT REGLT_NO, REGLT_DE FROM N_IS_PENALTY;")
ispe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11277810 entries, 0 to 11277809
Data columns (total 2 columns):
 #   Column    Dtype
---  ------    -----
 0   REGLT_NO  int64
 1   REGLT_DE  int64
dtypes: int64(2)
memory usage: 172.1 MB


In [43]:
ispe_ch_col = {
    'REGLT_NO':'적발번호', 
    'REGLT_DE':'단속일', 
}
isper = ispe.rename(columns=ispe_ch_col)
isper.columns

Index(['적발번호', '단속일'], dtype='object')

## [❗] 운행제한 단속정보(N_US_DISCLOSURE)

In [44]:
# # 14.2s
# usdis = pd.read_csv(os.path.join(ods_fold, 'ODS_N_US_DISCLOSURE.csv'), low_memory=False, usecols=['NO', 'VIN', 'REG_SIDO_CD', 'REG_SIGNGU_CD', 'DISCL_TY', 'REGLT_AREA_CD'])
# usdis_ch_dict = {
#     'NO':'번호', 
#     'VIN':'차대번호', 
#     'REG_SIDO_CD':'등록시도코드', 
#     'REG_SIGNGU_CD':'등록시군구코드', 
#     'DISCL_TY':'적발유형', 
#     'REGLT_AREA_CD':'단속지역코드', 
# }
# usdisr = usdis.rename(columns=usdis_ch_dict)
# usdisr.columns

In [45]:
# sql = 'select "no", VIN, DISCL_TY, REGLT_AREA_CD, REG_SIDO_CD, REG_SIGNGU_CD from vsysd.n_us_disclosure'
# cur.execute(sql)
# usdis = pd.DataFrame(cur.fetchall())
# usdis.columns = [desc[0].upper() for desc in cur.description]
# usdis_ch_dict = {
#     'NO':'번호', 
#     'VIN':'차대번호', 
#     'REG_SIDO_CD':'등록시도코드', 
#     'REG_SIGNGU_CD':'등록시군구코드', 
#     'DISCL_TY':'적발유형', 
#     'REGLT_AREA_CD':'단속지역코드', 
# }
# usdisr = usdis.rename(columns=usdis_ch_dict)
# usdisr.columns

In [46]:
usdis = wd.export_to_pandas('SELECT "NO", VIN, DISCL_TY, REGLT_AREA_CD, REG_SIDO_CD, REG_SIGNGU_CD FROM N_US_DISCLOSURE;')
usdis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3151659 entries, 0 to 3151658
Data columns (total 6 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   NO             int64 
 1   VIN            object
 2   DISCL_TY       object
 3   REGLT_AREA_CD  int64 
 4   REG_SIDO_CD    int64 
 5   REG_SIGNGU_CD  int64 
dtypes: int64(4), object(2)
memory usage: 144.3+ MB


In [47]:
usdis_ch_dict = {
    'NO':'번호', 
    'VIN':'차대번호', 
    'REG_SIDO_CD':'등록시도코드', 
    'REG_SIGNGU_CD':'등록시군구코드', 
    'DISCL_TY':'적발유형', 
    'REGLT_AREA_CD':'단속지역코드', 
}
usdisr = usdis.rename(columns=usdis_ch_dict)
usdisr.columns

Index(['번호', '차대번호', '적발유형', '단속지역코드', '등록시도코드', '등록시군구코드'], dtype='object')

In [48]:
# DISCL_TY(적발유형) : S(일반 -> 상시), H(고농도)

## [❗] N_US_PENALTY

In [49]:
# # 0.5s
# uspe = pd.read_csv(os.path.join(ods_fold, 'N_US_PENALTY.csv'), low_memory=False, sep='|', usecols=['NO', 'REGLT_CNT', 'REGLT_YM'])
# uspe_ch_dict = {
#     'NO':'번호', 
#     'REGLT_CNT':'적발건수', 
#     'REGLT_YM':'적발년월'
# }
# usper = uspe.rename(columns=uspe_ch_dict)
# usper.columns

In [50]:
# sql = 'select "no", REGLT_CNT, REGLT_YM FROM from vsysd.n_us_penalty'
# cur.execute(sql)
# uspe = pd.DataFrame(cur.fetchall())
# uspe.columns = [desc[0].upper() for desc in cur.description]
# uspe_ch_dict = {
#     'NO':'번호', 
#     'REGLT_CNT':'적발건수', 
#     'REGLT_YM':'적발년월'
# }
# usper = uspe.rename(columns=uspe_ch_dict)
# usper.columns

In [51]:
uspe = wd.export_to_pandas('SELECT "NO", REGLT_CNT, REGLT_YM FROM N_US_PENALTY;')
uspe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 444663 entries, 0 to 444662
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   NO         444663 non-null  int64
 1   REGLT_CNT  444663 non-null  int64
 2   REGLT_YM   444663 non-null  int64
dtypes: int64(3)
memory usage: 10.2 MB


In [52]:
uspe_ch_dict = {
    'NO':'번호', 
    'REGLT_CNT':'적발건수', 
    'REGLT_YM':'적발년월'
}
usper = uspe.rename(columns=uspe_ch_dict)
usper.columns

Index(['번호', '적발건수', '적발년월'], dtype='object')

## [❗] 등록이력(CEG_CAR_HISTORY_MIG)

In [53]:
# # 1m 13.2s
# his = ws.export_to_pandas("SELECT VHCL_ERSR_YN, CHNG_DE, VHMNO FROM CEG_CAR_HISTORY_MIG;")
# his_ch_col = {
#     'VHCL_ERSR_YN':'차량말소YN', 
#     'CHNG_DE':'변경일자',
#     'VHMNO':'차량관리번호'
# }
# hisr = his.rename(columns=his_ch_col)
# hisr.columns

In [54]:
# sql = 'select VHCL_ERSR_YN, CHNG_DE, VHMNO from vsysd.ceg_car_history_mig'
# cur.execute(sql)
# his = pd.DataFrame(cur.fetchall())
# his.columns = [desc[0].upper() for desc in cur.description]
# cur.close()
# conn.close()
# his_ch_col = {
#     'VHCL_ERSR_YN':'차량말소YN', 
#     'CHNG_DE':'변경일자',
#     'VHMNO':'차량관리번호'
# }
# hisr = his.rename(columns=his_ch_col)
# hisr.columns

In [55]:
his = wd.export_to_pandas("SELECT VHCL_ERSR_YN, CHNG_DE, VHMNO FROM CEG_CAR_HISTORY_MIG;")
his.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69241518 entries, 0 to 69241517
Data columns (total 3 columns):
 #   Column        Dtype  
---  ------        -----  
 0   VHCL_ERSR_YN  object 
 1   CHNG_DE       float64
 2   VHMNO         object 
dtypes: float64(1), object(2)
memory usage: 1.5+ GB


In [56]:
his_ch_col = {
    'VHCL_ERSR_YN':'차량말소YN', 
    'CHNG_DE':'변경일자',
    'VHMNO':'차량관리번호'
}
hisr = his.rename(columns=his_ch_col)
hisr.columns

Index(['차량말소YN', '변경일자', '차량관리번호'], dtype='object')

# 전처리

## 중복 차대번호 제거

In [57]:
carr.shape, len(carr['차대번호'].unique())

((2314337, 10), 2314288)

In [58]:
carr['최초등록일자'] = pd.to_numeric(carr['최초등록일자'], errors='coerce')
carr = carr.sort_values('최초등록일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
carr.shape

(2314288, 10)

## 차종 코드 변환

In [59]:
cd_dict = {
    'A31M':'이륜', 
    'A31P':'승용', 
    'A31S':'특수', 
    'A31T':'화물', 
    'A31V':'승합'
}
carr['차종'] = carr['차종'].replace(cd_dict)
carr['차종'].unique()

array(['화물', '특수', '승용', '승합'], dtype=object)

## 용도 코드 변환

In [60]:
purps_dict = {
    'A08P':'개인용', 
    'A08B':'영업용', 
    'A08O':'관용',
}
carr['용도'] = carr['용도'].replace(purps_dict)
carr['용도'].unique()

array(['개인용', '영업용', '관용'], dtype=object)

## 등록정보 말소 제거

In [61]:
carm = carr[carr['차량말소YN'] == 'N'].reset_index(drop=True)
carm.shape

(973987, 10)

## 연료 코드 변환

In [62]:
fuel_dict = {
    'A90GS':'휘발유', 
    'A91DS':'경유',
    'A92LP':'LPG(액화석유가스)', 
    'A90GH':'휘발유 하이브리드', 
    'A93EV':'전기', 
    'A91DH':'경유 하이브리드', 
    'A92CN':'CNG(압축천연가스)', 
    'A93HD':'수소', 
    'A92LH':'LPG 하이브리드', 
    'A94OT':'기타연료', 
    'A92CH':'CNG 하이브리드',
    'A90AC':'알코올', 
    'A93SH':'태양열', 
    'A91KS':'등유', 
    'A92LN':'LNG(액화천연가스)', 
    'A90PH':'플러그인 하이브리드', 
}
srcr['연료'] = srcr['연료'].replace(fuel_dict)
srcr['연료'].unique()

array(['휘발유', '경유', '기타연료', 'LPG(액화석유가스)', nan, '휘발유 하이브리드',
       'CNG(압축천연가스)', '알코올', '등유', '전기', '태양열', '경유 하이브리드', 'LPG 하이브리드',
       '수소', 'LNG(액화천연가스)', 'CNG 하이브리드'], dtype=object)

## 차종유형 코드 변환

In [63]:
ty_dict = {
    'A30C':'경형', 
    'A30L':'대형', 
    'A30M':'중형', 
    'A30S':'소형',
}
srcr['차종유형'] = srcr['차종유형'].replace(ty_dict)
srcr['차종유형'].unique()

array(['대형', '중형', '소형', '경형', nan], dtype=object)

## 등록&제원 병합

In [64]:
# 19.4s
cs = carm.merge(srcr, on='제원관리번호', how='left')
cs.shape

(973987, 12)

In [65]:
cse = carr.merge(srcr, on='제원관리번호', how='left')
cse.shape

(2314288, 12)

## 저감장치구분 코드 변환

In [66]:
# 4m 14.3s
attr['저감장치구분'].unique()

array(['A1001', 'A1003', 'A1005', 'A1002', 'A1006', 'A1007', 'A1004'],
      dtype=object)

In [67]:
# 27.5s
rdcdvc_dict = {
    'A1001':'1종', 
    'A1002':'2종', 
    'A1003':'3종', 
    'A1004':'1종+SCR', 
    'A1005':'엔진개조', 
    'A1006':'엔진교체',
    'A1007':'삼원촉매',
}
attr['저감장치구분'] = attr['저감장치구분'].replace(rdcdvc_dict)
attr['저감장치구분'].unique()

array(['1종', '3종', '엔진개조', '2종', '엔진교체', '삼원촉매', '1종+SCR'], dtype=object)

## 저감장치 부착 유무

In [68]:
attr.loc[(attr['저감장치구분'] == '1종') | (attr['저감장치구분'] == '1종+SCR'), 'DPF_YN'] = '유'

In [69]:
attr['DPF_YN'].value_counts(dropna=False)

DPF_YN
NaN    533631
유      482310
Name: count, dtype: int64

In [70]:
attr.shape, len(attr['차대번호'].unique())

((1015941, 3), 1014369)

In [71]:
attr = attr.sort_values('DPF_YN').drop_duplicates('차대번호').reset_index(drop=True)
attr.shape

(1014369, 3)

## 등록&제원&저감이력 병합

In [72]:
# 1.7s
csa = cs.merge(attr[['차대번호', 'DPF_YN']], on='차대번호', how='left')
csa.shape

(973987, 13)

## 지역정보 병합

In [73]:
csa['법정동코드'] = csa['법정동코드'].astype('str')
csa['법정동코드'] = csa['법정동코드'].str[:5] + '00000'
csa['법정동코드'] = pd.to_numeric(csa['법정동코드'])

In [74]:
csac = csa.merge(coder, on='법정동코드', how='left')

In [75]:
csac['시도'].isnull().sum()

0

## 조기폐차 정보들 병합

In [76]:
elp = pd.concat([aear, lgvr], ignore_index=True)
elp.shape

(1200370, 4)

In [77]:
elp.shape, len(elp['차대번호'].unique())

((1200370, 4), 1125610)

In [78]:
elpm = elp.sort_values('조기폐차최종승인YN', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
elpm.shape

(1125610, 4)

In [79]:
elpm = elpm[elpm['조기폐차최종승인YN'] == 'Y'].reset_index(drop=True)
elpm.shape

(887431, 4)

## 조기폐차 정보 추가

In [80]:
csac.shape

(973987, 17)

In [81]:
df = csac.merge(elpm, on='차대번호', how='left')
df.shape

(973987, 20)

In [82]:
df.head()

,차대번호,법정동코드,차량말소YN,제원관리번호,차량연식,차종,용도,최초등록일자,제작일자,차량관리번호,연료,차종유형,DPF_YN,시도코드,시군구코드,시도,시군구,조기폐차상태코드,조기폐차최종승인YN,말소일자
0,KL3C3AHF27K000073,4785000000,N,A0310007202023307,2007,화물,개인용,20230613.00,20070627.00,KL3C3AHF27K000073-01,경유,대형,NaN,47,850,경상북도,칠곡군,NaN,NaN,NaN
1,KMFGA17PP7C071816,4427000000,N,00034427000023223,2007,화물,개인용,20230221.00,20061231.00,KMFGA17PP7C071816-01,경유,중형,NaN,44,270,충청남도,당진시,NaN,NaN,NaN
2,KL3C3AHF27K000082,4481000000,N,A0310007202023307,2007,화물,개인용,20220711.00,20070627.00,KL3C3AHF27K000082-01,경유,대형,NaN,44,810,충청남도,예산군,NaN,NaN,NaN
3,KRGLAAC3K3B019534,2714000000,N,00034427000033222,2003,화물,개인용,20220414.00,20021231.00,KRGLAAC3K3B019534-01,경유,중형,NaN,27,140,대구광역시,동구,NaN,NaN,NaN
4,KL3C3AHF27K000068,4684000000,N,A0310007202023307,2007,화물,개인용,20220413.00,20070618.00,KL3C3AHF27K000068-01,경유,대형,NaN,46,840,전라남도,무안군,NaN,NaN,NaN


## 비상시 및 계절제 병합

In [83]:
isdisr.shape, isisr.shape, isper.shape

((108436093, 6), (2368, 3), (11277810, 2))

In [84]:
isdisr.columns, isisr.columns, isper.columns

(Index(['적발번호', '발령번호', '차대번호', '등록시도코드', '등록시군구코드', '적발지역코드'], dtype='object'),
 Index(['발령번호', '유형기준아이디', '농도기준아이디'], dtype='object'),
 Index(['적발번호', '단속일'], dtype='object'))

In [85]:
isdisr.shape

(108436093, 6)

In [86]:
# 41.3s
isdp= isdisr.merge(isper, on='적발번호', how='left')
isdp.shape

(108436093, 7)

In [87]:
# 9.5s
isdpi = isdp.merge(isisr, on='발령번호', how='left')
isdpi.shape

(108436093, 9)

### 지역정보 추가

In [88]:
coder_dup = coder.sort_values('법정동코드', ascending=True).drop_duplicates(['시도코드', '시군구코드']).reset_index(drop=True)

In [89]:
coder_dup.head()

,법정동코드,시도코드,시군구코드,시도,시군구
0,1100000000,11,0,서울특별시,NaN
1,1111000000,11,110,서울특별시,종로구
2,1114000000,11,140,서울특별시,중구
3,1117000000,11,170,서울특별시,용산구
4,1120000000,11,200,서울특별시,성동구


In [90]:
isdpi.columns

Index(['적발번호', '발령번호', '차대번호', '등록시도코드', '등록시군구코드', '적발지역코드', '단속일', '유형기준아이디',
       '농도기준아이디'],
      dtype='object')

In [91]:
isdpi[isdpi['등록시도코드'] == 42].shape, isdpi[isdpi['등록시도코드'] == 51].shape

((1030435, 9), (0, 9))

In [92]:
isdpi.loc[isdpi['등록시도코드'] == 42, '등록시도코드'] = 51 # !!! 수정

In [93]:
# 28.4s
is_total1 = isdpi.merge(coder_dup[['시도코드', '시군구코드', '시도', '시군구']], left_on=['등록시도코드', '등록시군구코드'], right_on=['시도코드', '시군구코드'], how='left')
is_total1 = is_total1.drop(['시도코드', '시군구코드'], axis=1)
is_total1.shape

(108436093, 11)

In [94]:
is_total1 = is_total1.rename(columns={'시도':'등록시도', '시군구':'등록시군구'})

In [95]:
is_total1['등록시도'].isnull().sum()

0

In [96]:
# 1m 19.2s
is_total1['적발지역코드'] = is_total1['적발지역코드'].astype('str')
is_total1['적발시도코드'] = is_total1['적발지역코드'].str[:2]
is_total1['적발시군구코드'] = is_total1['적발지역코드'].str[2:5]
is_total1[['적발시도코드', '적발시군구코드']] = is_total1[['적발시도코드', '적발시군구코드']].astype('int')

In [97]:
is_total1[is_total1['적발시도코드'] == 42].shape, is_total1[is_total1['적발시도코드'] == 51].shape

((600309, 13), (0, 13))

In [98]:
is_total1[is_total1['적발시도코드'] == 36].shape, is_total1.loc[(is_total1['적발시도코드'] == 36) & (is_total1['적발시군구코드'] == 110)].shape

((1297863, 13), (0, 13))

In [99]:
is_total1.loc[is_total1['적발시도코드'] == 42, '적발시도코드'] = 51 # !!! 수정
is_total1.loc[is_total1['적발시도코드'] == 36, '적발시군구코드'] = 110 # !!! 수정

In [100]:
coder_dup[['시도코드', '시군구코드']].dtypes

시도코드     int64
시군구코드    int64
dtype: object

In [101]:
# 16.5s
is_total = is_total1.merge(coder_dup[['시도코드', '시군구코드', '시도', '시군구']], left_on=['적발시도코드', '적발시군구코드'], right_on=['시도코드', '시군구코드'], how='left')

In [102]:
is_total['시도'].isnull().sum()

0

In [103]:
is_total.columns

Index(['적발번호', '발령번호', '차대번호', '등록시도코드', '등록시군구코드', '적발지역코드', '단속일', '유형기준아이디',
       '농도기준아이디', '등록시도', '등록시군구', '적발시도코드', '적발시군구코드', '시도코드', '시군구코드', '시도',
       '시군구'],
      dtype='object')

In [104]:
# 1m 12.8s
is_total = is_total.drop(['시도코드', '시군구코드'], axis=1)
is_total = is_total.rename(columns={'시도':'적발시도', '시군구':'적발시군구'})
is_total.head()

,적발번호,발령번호,차대번호,등록시도코드,등록시군구코드,적발지역코드,단속일,유형기준아이디,농도기준아이디,등록시도,등록시군구,적발시도코드,적발시군구코드,적발시도,적발시군구
0,51565757,1821,KMFPA18TP5C001735,41,170,28000,NaN,T001,C012,경기도,안양시,28,0,인천광역시,NaN
1,51565758,1821,KNAJE55135K039713,28,260,28000,NaN,T001,C012,인천광역시,서구,28,0,인천광역시,NaN
2,51652557,1821,KL5UL42PD7U000105,11,650,11000,NaN,T001,C012,서울특별시,서초구,11,0,서울특별시,NaN
3,51652540,1821,KMFGA17PP6C041875,41,460,11000,NaN,T001,C012,경기도,용인시,11,0,서울특별시,NaN
4,51652532,1821,KMFZCS7JP6U175495,41,170,11000,NaN,T001,C012,경기도,안양시,11,0,서울특별시,NaN


## 상시 병합

In [105]:
usdisr.head()

,번호,차대번호,적발유형,단속지역코드,등록시도코드,등록시군구코드
0,8389408,KMFZCD7JP5U081362,S,11000,11,140
1,8389410,KN3HNP6D15K094646,S,11000,11,140
2,8389713,KNAMA75432S302063,S,11000,11,140
3,8389714,KMFLA19RPYC015091,S,11000,11,140
4,8389716,KMHWP81JP5U664886,S,11000,11,140


In [106]:
usper.head()

,번호,적발건수,적발년월
0,8114417,1,201901
1,6933701,4,201901
2,6938411,10,201901
3,8114173,1,201901
4,6938670,14,201901


In [107]:
usdisr.shape

(3151659, 6)

In [108]:
# 1s
usdp = usdisr.merge(usper, on='번호', how='left')
usdp.shape

(3151659, 8)

### 지역정보 추가

In [109]:
usdp[usdp['등록시도코드'] == 42].shape, usdp[usdp['등록시도코드'] == 51].shape

((32447, 8), (0, 8))

In [110]:
usdp[usdp['등록시도코드'] == 36].shape, usdp.loc[(usdp['등록시도코드'] == 36) & (usdp['등록시군구코드'] == 110)].shape

((2919, 8), (35, 8))

In [111]:
usdp[['등록시도코드', '등록시군구코드']].dtypes

등록시도코드     int64
등록시군구코드    int64
dtype: object

In [112]:
usdp.loc[usdp['등록시도코드'] == 42, '등록시도코드'] = 51 # !!! 수정
usdp.loc[usdp['등록시도코드'] == 36, '등록시군구코드'] = 110 # !!! 수정

In [113]:
us_total1 = usdp.merge(coder_dup, left_on=['등록시도코드', '등록시군구코드'], right_on=['시도코드', '시군구코드'], how='left')
us_total1.head()

,번호,차대번호,적발유형,단속지역코드,등록시도코드,등록시군구코드,적발건수,적발년월,법정동코드,시도코드,시군구코드,시도,시군구
0,8389408,KMFZCD7JP5U081362,S,11000,11,140,NaN,NaN,1114000000,11,140,서울특별시,중구
1,8389410,KN3HNP6D15K094646,S,11000,11,140,NaN,NaN,1114000000,11,140,서울특별시,중구
2,8389713,KNAMA75432S302063,S,11000,11,140,1.00,201908.00,1114000000,11,140,서울특별시,중구
3,8389714,KMFLA19RPYC015091,S,11000,11,140,1.00,201908.00,1114000000,11,140,서울특별시,중구
4,8389716,KMHWP81JP5U664886,S,11000,11,140,NaN,NaN,1114000000,11,140,서울특별시,중구


In [114]:
us_total1 = us_total1.drop(['시도코드', '시군구코드'], axis=1)
us_total1 = us_total1.rename(columns={'시도':'등록시도', '시군구':'등록시군구'}) # !!! 수정(2023.09.01)
us_total1.head()

,번호,차대번호,적발유형,단속지역코드,등록시도코드,등록시군구코드,적발건수,적발년월,법정동코드,등록시도,등록시군구
0,8389408,KMFZCD7JP5U081362,S,11000,11,140,NaN,NaN,1114000000,서울특별시,중구
1,8389410,KN3HNP6D15K094646,S,11000,11,140,NaN,NaN,1114000000,서울특별시,중구
2,8389713,KNAMA75432S302063,S,11000,11,140,1.00,201908.00,1114000000,서울특별시,중구
3,8389714,KMFLA19RPYC015091,S,11000,11,140,1.00,201908.00,1114000000,서울특별시,중구
4,8389716,KMHWP81JP5U664886,S,11000,11,140,NaN,NaN,1114000000,서울특별시,중구


In [115]:
us_total1['등록시도'].isnull().sum()

0

In [117]:
# !!! 수정(2023.09.01)
# 4s
us_total1['단속지역코드'] = us_total1['단속지역코드'].astype('str')
us_total1['단속시도코드'] = us_total1['단속지역코드'].str[:2]
us_total1['단속시군구코드'] = us_total1['단속지역코드'].str[2:5]
us_total1[['단속시도코드', '단속시군구코드']] = us_total1[['단속시도코드', '단속시군구코드']].astype('int')

In [118]:
us_total1[us_total1['단속시도코드'] == 42].shape, us_total1[us_total1['단속시도코드'] == 51].shape

((0, 13), (0, 13))

In [119]:
us_total1[us_total1['단속시도코드'] == 36].shape, us_total1.loc[(us_total1['단속시도코드'] == 36) & (us_total1['단속시군구코드'] == 110)].shape

((0, 13), (0, 13))

In [ ]:
# us_total1.loc[us_total1['단속시도코드'] == 42, '단속시도코드'] = 51
# us_total1.loc[us_total1['단속시도코드'] == 36, '단속시군구코드'] = 110

In [120]:
coder_dup[['시도코드', '시군구코드']].dtypes

시도코드     int64
시군구코드    int64
dtype: object

In [121]:
# !!! 수정(2023.09.01)
# 2s
us_total = us_total1.merge(coder_dup[['시도코드', '시군구코드', '시도', '시군구']], left_on=['단속시도코드', '단속시군구코드'], right_on=['시도코드', '시군구코드'], how='left')
us_total = us_total.drop(['시도코드', '시군구코드'], axis=1)
us_total = us_total.rename(columns={'시도':'단속시도', '시군구':'단속시군구'})

In [122]:
us_total['단속시도'].isnull().sum()

0

In [124]:
us_total.head()

,번호,차대번호,적발유형,단속지역코드,등록시도코드,등록시군구코드,적발건수,적발년월,법정동코드,등록시도,등록시군구,단속시도코드,단속시군구코드,단속시도,단속시군구
0,8389408,KMFZCD7JP5U081362,S,11000,11,140,NaN,NaN,1114000000,서울특별시,중구,11,0,서울특별시,NaN
1,8389410,KN3HNP6D15K094646,S,11000,11,140,NaN,NaN,1114000000,서울특별시,중구,11,0,서울특별시,NaN
2,8389713,KNAMA75432S302063,S,11000,11,140,1.00,201908.00,1114000000,서울특별시,중구,11,0,서울특별시,NaN
3,8389714,KMFLA19RPYC015091,S,11000,11,140,1.00,201908.00,1114000000,서울특별시,중구,11,0,서울특별시,NaN
4,8389716,KMHWP81JP5U664886,S,11000,11,140,NaN,NaN,1114000000,서울특별시,중구,11,0,서울특별시,NaN


In [125]:
is_total.shape, us_total.shape

((108436093, 15), (3151659, 15))

## 등록(말소 유지) & 제원 병합

In [126]:
# 10.3s
cse = carr.merge(srcr, on='제원관리번호', how='left')

In [127]:
cse.columns

Index(['차대번호', '법정동코드', '차량말소YN', '제원관리번호', '차량연식', '차종', '용도', '최초등록일자',
       '제작일자', '차량관리번호', '연료', '차종유형'],
      dtype='object')

## 등록&제원&이력 병합

In [128]:
# 2m 6.0s
ersr = cse.merge(hisr, on='차량관리번호', how='left')

In [129]:
ersr.columns

Index(['차대번호', '법정동코드', '차량말소YN_x', '제원관리번호', '차량연식', '차종', '용도', '최초등록일자',
       '제작일자', '차량관리번호', '연료', '차종유형', '차량말소YN_y', '변경일자'],
      dtype='object')

In [130]:
# 1. 등록 차량말소와 등록이력 차량 말소 둘 모두 해당되는 데이터 추출
ersr = ersr.loc[(ersr['차량말소YN_x'] == 'Y') & (ersr['차량말소YN_y'] == 'Y')].reset_index(drop=True)

In [131]:
# 2. 변경일자 기준 최신 데이터만 남기고 차대번호 중복 제거
ersr = ersr.sort_values('변경일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)

In [132]:
# 3. 변경일자 2019.01.01 이후만 추출
ersr = ersr[ersr['변경일자'] >= 20190101].reset_index(drop=True)

## 등록&제원&이력&법정동코드 병합(errc)

In [133]:
ersr['법정동코드'] = ersr['법정동코드'].astype('str')
ersr['법정동코드'] = ersr['법정동코드'].str[:5] + '00000'
ersr['법정동코드'] = pd.to_numeric(ersr['법정동코드'])

In [134]:
# 매칭 후 '시도' 빈값 없음
errc = ersr.merge(coder, on='법정동코드', how='left')

In [135]:
errc['시도'].isnull().sum()

0

# Load

## 5등급 지역별 조기폐차(STD_BD_GRD5_ELPDSRC_CURSTT)

In [136]:
df.columns

Index(['차대번호', '법정동코드', '차량말소YN', '제원관리번호', '차량연식', '차종', '용도', '최초등록일자',
       '제작일자', '차량관리번호', '연료', '차종유형', 'DPF_YN', '시도코드', '시군구코드', '시도', '시군구',
       '조기폐차상태코드', '조기폐차최종승인YN', '말소일자'],
      dtype='object')

In [137]:
# 8.6s
df1 = we.export_to_pandas("SELECT * FROM STD_BD_GRD5_ELPDSRC;")
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   RGN         68 non-null     object 
 1   YR          68 non-null     int64  
 2   VHCL_MKCNT  68 non-null     int64  
 3   VHCL_REDE   68 non-null     float64
 4   ELPDSRC     68 non-null     float64
 5   NTRL_DCLN   68 non-null     float64
dtypes: float64(3), int64(2), object(1)
memory usage: 3.3+ KB


In [138]:
df1 = df1.dropna()
df1.shape

(68, 6)

## 5등급 저공해 미조치(STD_BD_GRD5_LEM_N_MOD)

In [139]:
# 8.6s
no_dpf = we.export_to_pandas("SELECT * FROM STD_BD_GRD5_LEM_N_MOD;")
no_dpf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   RGN         12 non-null     object
 1   SEASON      12 non-null     object
 2   VHCL_MKCNT  12 non-null     int64 
 3   LOAD_DT     12 non-null     int64 
dtypes: int64(2), object(2)
memory usage: 512.0+ bytes


In [140]:
no_dpf.head()

,RGN,SEASON,VHCL_MKCNT,LOAD_DT
0,수도권,2차계절제_시작,539748,20230810
1,수도권,2차계절제_종료,515615,20230810
2,수도권,3차계절제_시작,459110,20230810
3,수도권,3차계절제_종료,436921,20230810
4,수도권,4차계절제_시작,394533,20230810


# 분석

## 5등급 지역별 조기폐차 현황

In [141]:
dfm = df.copy()

In [142]:
dfm['최초등록일자'] = dfm['최초등록일자'].astype('str')
dfm['최초등록일자_년'] = dfm['최초등록일자'].str[:4]
dfm['최초등록일자_월'] = dfm['최초등록일자'].str[4:6]
dfm['최초등록일자_일'] = dfm['최초등록일자'].str[6:8]

In [143]:
errc['변경일자'] = errc['변경일자'].astype('str')
errc['변경일자_년'] = errc['변경일자'].str[:4]
errc['변경일자_월'] = errc['변경일자'].str[4:6]
errc['변경일자_일'] = errc['변경일자'].str[6:8]

## 시도, 연도별 차량 현황 분석

In [144]:
int(datetime.today().strftime("%Y"))

2023

In [145]:
# 2022년 차량 대수
grp1 = dfm.groupby(['시도'], as_index=False)['차대번호'].count()
grp1 = grp1.rename(columns={'차대번호':'차량대수'})
# year = 2022
year = int(datetime.today().strftime("%Y")) # !!! 수정(2023.08.31)
grp1['연도'] = f'{year}'
grp1 = grp1[['연도', '시도', '차량대수']]

In [146]:
grp1.head()

,연도,시도,차량대수
0,2023,강원특별자치도,39205
1,2023,경기도,197307
2,2023,경상남도,87566
3,2023,경상북도,100815
4,2023,광주광역시,21863


In [147]:
# 4년간 차량 통계 기본 데이터셋
yr_list, ctpv_list = [], []
for ctpv in grp1['시도'].unique():
    for yr in range(year - 3, year + 1): # !!! 수정(2023.08.31)
        yr_list.append(str(yr))
        ctpv_list.append(ctpv)
base = pd.DataFrame({'연도':yr_list, '시도':ctpv_list})

In [148]:
base.head()

,연도,시도
0,2020,강원특별자치도
1,2021,강원특별자치도
2,2022,강원특별자치도
3,2023,강원특별자치도
4,2020,경기도


In [149]:
# 연도별 등록대수
grp2 = dfm.groupby(['최초등록일자_년', '시도'], as_index=False)['차대번호'].count()
grp2 = grp2.rename(columns={'최초등록일자_년':'연도', '차대번호':'등록대수'})
grp2

,연도,시도,등록대수
0,1920,경기도,2
1,1920,서울특별시,1
2,1942,강원특별자치도,2
3,1943,강원특별자치도,3
4,1944,강원특별자치도,5
...,...,...,...
831,2023,경상북도,1
832,2023,충청남도,1
833,nan,대구광역시,4
834,nan,서울특별시,1


In [150]:
# 연도별 말소대수
grp3 = errc.groupby(['변경일자_년', '시도'], as_index=False)['차대번호'].count()
grp3 = grp3.rename(columns={'변경일자_년':'연도', '차대번호':'말소대수'})
grp3

,연도,시도,말소대수
0,2019,경기도,73287
1,2019,경상남도,17541
2,2019,경상북도,19635
3,2019,광주광역시,5895
4,2019,대구광역시,10822
...,...,...,...
76,2023,전라남도,7859
77,2023,전라북도,5575
78,2023,제주특별자치도,3145
79,2023,충청남도,6528


In [151]:
base1 = base.merge(grp1, on=['연도', '시도'], how='left')
base1

,연도,시도,차량대수
0,2020,강원특별자치도,NaN
1,2021,강원특별자치도,NaN
2,2022,강원특별자치도,NaN
3,2023,강원특별자치도,39205.00
4,2020,경기도,NaN
...,...,...,...
63,2023,충청남도,53434.00
64,2020,충청북도,NaN
65,2021,충청북도,NaN
66,2022,충청북도,NaN


In [152]:
base2 = base1.merge(grp2, on=['연도', '시도'], how='left')
base2

,연도,시도,차량대수,등록대수
0,2020,강원특별자치도,NaN,1.00
1,2021,강원특별자치도,NaN,NaN
2,2022,강원특별자치도,NaN,NaN
3,2023,강원특별자치도,39205.00,NaN
4,2020,경기도,NaN,1.00
...,...,...,...,...
63,2023,충청남도,53434.00,1.00
64,2020,충청북도,NaN,NaN
65,2021,충청북도,NaN,NaN
66,2022,충청북도,NaN,NaN


In [153]:
base3 = base2.merge(grp3, on=['연도', '시도'], how='left')
base3

,연도,시도,차량대수,등록대수,말소대수
0,2020,강원특별자치도,NaN,1.00,NaN
1,2021,강원특별자치도,NaN,NaN,NaN
2,2022,강원특별자치도,NaN,NaN,NaN
3,2023,강원특별자치도,39205.00,NaN,1279.00
4,2020,경기도,NaN,1.00,95909.00
...,...,...,...,...,...
63,2023,충청남도,53434.00,1.00,6528.00
64,2020,충청북도,NaN,NaN,23584.00
65,2021,충청북도,NaN,NaN,19528.00
66,2022,충청북도,NaN,NaN,9000.00


In [154]:
base3[['차량대수', '등록대수', '말소대수']].isnull().sum()

차량대수    51
등록대수    47
말소대수     3
dtype: int64

In [155]:
base3[['차량대수', '등록대수', '말소대수']] = base3[['차량대수', '등록대수', '말소대수']].fillna(0)

In [156]:
n = len(base3['연도'].unique())
for i in range(base3.shape[0] // n):
    for j in range(2, n+1):
        base3.loc[(i+1)*n - j, '차량대수'] = base3.loc[(i+1)*n - (j-1), '차량대수'] + base3.loc[(i+1)*n - (j-1), '말소대수'] - base3.loc[(i+1)*n - (j-1), '등록대수']

In [157]:
base3

,연도,시도,차량대수,등록대수,말소대수
0,2020,강원특별자치도,40484.00,1.00,0.00
1,2021,강원특별자치도,40484.00,0.00,0.00
2,2022,강원특별자치도,40484.00,0.00,0.00
3,2023,강원특별자치도,39205.00,0.00,1279.00
4,2020,경기도,314192.00,1.00,95909.00
...,...,...,...,...,...
63,2023,충청남도,53434.00,1.00,6528.00
64,2020,충청북도,68661.00,0.00,23584.00
65,2021,충청북도,49133.00,0.00,19528.00
66,2022,충청북도,40133.00,0.00,9000.00


In [158]:
dfm.columns

Index(['차대번호', '법정동코드', '차량말소YN', '제원관리번호', '차량연식', '차종', '용도', '최초등록일자',
       '제작일자', '차량관리번호', '연료', '차종유형', 'DPF_YN', '시도코드', '시군구코드', '시도', '시군구',
       '조기폐차상태코드', '조기폐차최종승인YN', '말소일자', '최초등록일자_년', '최초등록일자_월', '최초등록일자_일'],
      dtype='object')

In [159]:
dfm['말소일자'] = dfm['말소일자'].astype('str')
dfm['말소일자_년'] = dfm['말소일자'].str[:4]
dfm['말소일자_월'] = dfm['말소일자'].str[4:6]
dfm['말소일자_일'] = dfm['말소일자'].str[6:8]

In [160]:
temp = pd.to_numeric(dfm['말소일자'], errors='coerce')
temp.describe()

count          6.00
mean    20183723.83
std        36788.53
min     20130201.00
25%     20157908.00
50%     20195262.50
75%     20210406.50
max     20220706.00
Name: 말소일자, dtype: float64

In [161]:
grp4 = dfm.loc[dfm['조기폐차최종승인YN'] == 'Y'].groupby(['말소일자_년'], as_index=False)['차대번호'].count()
grp4 = grp4.rename(columns={'말소일자_년':'연도', '차대번호':'조기폐차대수'})
grp4

,연도,조기폐차대수
0,2013,1
1,2015,1
2,2018,1
3,2021,2
4,2022,1


In [162]:
base4 = base3.merge(grp4, on='연도', how='left')
base4

,연도,시도,차량대수,등록대수,말소대수,조기폐차대수
0,2020,강원특별자치도,40484.00,1.00,0.00,NaN
1,2021,강원특별자치도,40484.00,0.00,0.00,2.00
2,2022,강원특별자치도,40484.00,0.00,0.00,1.00
3,2023,강원특별자치도,39205.00,0.00,1279.00,NaN
4,2020,경기도,314192.00,1.00,95909.00,NaN
...,...,...,...,...,...,...
63,2023,충청남도,53434.00,1.00,6528.00,NaN
64,2020,충청북도,68661.00,0.00,23584.00,NaN
65,2021,충청북도,49133.00,0.00,19528.00,2.00
66,2022,충청북도,40133.00,0.00,9000.00,1.00


In [163]:
base4['조기폐차대수'] = base4['조기폐차대수'].fillna(0)
base4 = base4.drop(['등록대수', '말소대수'], axis=1)
base4

,연도,시도,차량대수,조기폐차대수
0,2020,강원특별자치도,40484.00,0.00
1,2021,강원특별자치도,40484.00,2.00
2,2022,강원특별자치도,40484.00,1.00
3,2023,강원특별자치도,39205.00,0.00
4,2020,경기도,314192.00,0.00
...,...,...,...,...
63,2023,충청남도,53434.00,0.00
64,2020,충청북도,68661.00,0.00
65,2021,충청북도,49133.00,2.00
66,2022,충청북도,40133.00,1.00


In [164]:
n = len(base4['연도'].unique())
for i in range(base4.shape[0] // n):
    for j in range(n-1):
        base4.loc[i*4 + j+1, '감소대수'] = base4.loc[i*4 + j, '차량대수'] - base4.loc[i*4 + j+1, '차량대수']

In [165]:
base4

,연도,시도,차량대수,조기폐차대수,감소대수
0,2020,강원특별자치도,40484.00,0.00,NaN
1,2021,강원특별자치도,40484.00,2.00,0.00
2,2022,강원특별자치도,40484.00,1.00,0.00
3,2023,강원특별자치도,39205.00,0.00,1279.00
4,2020,경기도,314192.00,0.00,NaN
...,...,...,...,...,...
63,2023,충청남도,53434.00,0.00,6527.00
64,2020,충청북도,68661.00,0.00,NaN
65,2021,충청북도,49133.00,2.00,19528.00
66,2022,충청북도,40133.00,1.00,9000.00


In [166]:
base4['자연감소대수'] = base4['감소대수'] - base4['조기폐차대수']

In [167]:
base4

,연도,시도,차량대수,조기폐차대수,감소대수,자연감소대수
0,2020,강원특별자치도,40484.00,0.00,NaN,NaN
1,2021,강원특별자치도,40484.00,2.00,0.00,-2.00
2,2022,강원특별자치도,40484.00,1.00,0.00,-1.00
3,2023,강원특별자치도,39205.00,0.00,1279.00,1279.00
4,2020,경기도,314192.00,0.00,NaN,NaN
...,...,...,...,...,...,...
63,2023,충청남도,53434.00,0.00,6527.00,6527.00
64,2020,충청북도,68661.00,0.00,NaN,NaN
65,2021,충청북도,49133.00,2.00,19528.00,19526.00
66,2022,충청북도,40133.00,1.00,9000.00,8999.00


In [168]:
# 음수 존재하여 사용 못함
base4[base4['자연감소대수'] < 0].shape

(2, 6)

In [169]:
today_date = datetime.today().strftime("%Y%m%d")
today_date

'20230901'

In [170]:
df1.head()

,RGN,YR,VHCL_MKCNT,VHCL_REDE,ELPDSRC,NTRL_DCLN
0,서울,2019,199898,46949.67,31792.00,15157.67
1,서울,2020,161864,38034.00,23045.00,14989.00
2,서울,2021,138381,23483.00,8148.00,15335.00
3,서울,2022,120996,17385.00,2476.00,14909.00
4,인천,2019,90619,20596.67,16421.67,4175.00


In [171]:
df1['LOAD_DT'] = today_date

In [172]:
df1[['VHCL_REDE', 'ELPDSRC', 'NTRL_DCLN']] = df1[['VHCL_REDE', 'ELPDSRC', 'NTRL_DCLN']].round(0)

In [173]:
df1

,RGN,YR,VHCL_MKCNT,VHCL_REDE,ELPDSRC,NTRL_DCLN,LOAD_DT
0,서울,2019,199898,46950.00,31792.00,15158.00,20230901
1,서울,2020,161864,38034.00,23045.00,14989.00,20230901
2,서울,2021,138381,23483.00,8148.00,15335.00,20230901
3,서울,2022,120996,17385.00,2476.00,14909.00,20230901
4,인천,2019,90619,20597.00,16422.00,4175.00,20230901
...,...,...,...,...,...,...,...
63,경남,2022,98885,24199.00,14765.00,9434.00,20230901
64,제주,2019,44978,9340.00,5424.00,3915.00,20230901
65,제주,2020,37016,7962.00,4562.00,3400.00,20230901
66,제주,2021,29057,7959.00,4501.00,3458.00,20230901


In [174]:
STD_BD_GRD5_ELPDSRC_CURSTT = df1.copy()
STD_BD_GRD5_ELPDSRC_CURSTT.columns

Index(['RGN', 'YR', 'VHCL_MKCNT', 'VHCL_REDE', 'ELPDSRC', 'NTRL_DCLN',
       'LOAD_DT'],
      dtype='object')

In [175]:
STD_BD_GRD5_ELPDSRC_CURSTT.head()

,RGN,YR,VHCL_MKCNT,VHCL_REDE,ELPDSRC,NTRL_DCLN,LOAD_DT
0,서울,2019,199898,46950.00,31792.00,15158.00,20230901
1,서울,2020,161864,38034.00,23045.00,14989.00,20230901
2,서울,2021,138381,23483.00,8148.00,15335.00,20230901
3,서울,2022,120996,17385.00,2476.00,14909.00,20230901
4,인천,2019,90619,20597.00,16422.00,4175.00,20230901


In [176]:
try:
    STD_BD_GRD5_ELPDSRC_CURSTT['VHCL_MKCNT'] = STD_BD_GRD5_ELPDSRC_CURSTT['VHCL_MKCNT'].str.replace(',', '')
    STD_BD_GRD5_ELPDSRC_CURSTT['VHCL_REDE'] = STD_BD_GRD5_ELPDSRC_CURSTT['VHCL_REDE'].str.replace(',', '')
    STD_BD_GRD5_ELPDSRC_CURSTT['ELPDSRC'] = STD_BD_GRD5_ELPDSRC_CURSTT['ELPDSRC'].str.replace(',', '')
    STD_BD_GRD5_ELPDSRC_CURSTT['NTRL_DCLN'] = STD_BD_GRD5_ELPDSRC_CURSTT['NTRL_DCLN'].str.replace(',', '')
    STD_BD_GRD5_ELPDSRC_CURSTT['VHCL_MKCNT'] = pd.to_numeric(STD_BD_GRD5_ELPDSRC_CURSTT['VHCL_MKCNT'], errors='coerce')
    STD_BD_GRD5_ELPDSRC_CURSTT['VHCL_REDE'] = pd.to_numeric(STD_BD_GRD5_ELPDSRC_CURSTT['VHCL_REDE'], errors='coerce')
    STD_BD_GRD5_ELPDSRC_CURSTT['ELPDSRC'] = pd.to_numeric(STD_BD_GRD5_ELPDSRC_CURSTT['ELPDSRC'], errors='coerce')
    STD_BD_GRD5_ELPDSRC_CURSTT['NTRL_DCLN'] = pd.to_numeric(STD_BD_GRD5_ELPDSRC_CURSTT['NTRL_DCLN'], errors='coerce')
except:
    pass

In [177]:
STD_BD_GRD5_ELPDSRC_CURSTT.head()

,RGN,YR,VHCL_MKCNT,VHCL_REDE,ELPDSRC,NTRL_DCLN,LOAD_DT
0,서울,2019,199898,46950.00,31792.00,15158.00,20230901
1,서울,2020,161864,38034.00,23045.00,14989.00,20230901
2,서울,2021,138381,23483.00,8148.00,15335.00,20230901
3,서울,2022,120996,17385.00,2476.00,14909.00,20230901
4,인천,2019,90619,20597.00,16422.00,4175.00,20230901


In [178]:
STD_BD_GRD5_ELPDSRC_CURSTT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   RGN         68 non-null     object 
 1   YR          68 non-null     int64  
 2   VHCL_MKCNT  68 non-null     int64  
 3   VHCL_REDE   68 non-null     float64
 4   ELPDSRC     68 non-null     float64
 5   NTRL_DCLN   68 non-null     float64
 6   LOAD_DT     68 non-null     object 
dtypes: float64(3), int64(2), object(2)
memory usage: 3.8+ KB


### [출력] [D] STD_BD_GRD5_ELPDSRC_CURSTT

In [179]:
# expdf = STD_BD_GRD5_ELPDSRC_CURSTT
# table_nm = 'STD_BD_GRD5_ELPDSRC_CURSTT'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 5s
# we.import_from_pandas(expdf, table_nm)

In [180]:
# # 1.3s
# STD_BD_GRD5_ELPDSRC_CURSTT.to_csv(os.path.join(df3_fold, 'STD_BD_GRD5_ELPDSRC_CURSTT.csv'), index=False)

## 5등급 지역별 저공해미조치 차량현황

In [181]:
no_dpf['테이블생성일자'] = today_date

In [182]:
cdict = {
    '테이블생성일자':'LOAD_DT',
    '지역':'RGN',
    '구분':'SEASON',
    '차량대수':'VHCL_MKCNT', 
}

In [183]:
STD_BD_GRD5_LEM_N_MOD = no_dpf.rename(columns=cdict)
STD_BD_GRD5_LEM_N_MOD.columns

Index(['RGN', 'SEASON', 'VHCL_MKCNT', 'LOAD_DT', 'LOAD_DT'], dtype='object')

In [184]:
STD_BD_GRD5_LEM_N_MOD.head()

,RGN,SEASON,VHCL_MKCNT,LOAD_DT,LOAD_DT
0,수도권,2차계절제_시작,539748,20230810,20230901
1,수도권,2차계절제_종료,515615,20230810,20230901
2,수도권,3차계절제_시작,459110,20230810,20230901
3,수도권,3차계절제_종료,436921,20230810,20230901
4,수도권,4차계절제_시작,394533,20230810,20230901


In [185]:
try:
    STD_BD_GRD5_LEM_N_MOD['VHCL_MKCNT'] = STD_BD_GRD5_LEM_N_MOD['VHCL_MKCNT'].str.replace(',', '') # !!! 수정
    STD_BD_GRD5_LEM_N_MOD['VHCL_MKCNT'] = pd.to_numeric(STD_BD_GRD5_LEM_N_MOD['VHCL_MKCNT'], errors='coerce') # !!! 수정
except:
    pass

In [186]:
STD_BD_GRD5_LEM_N_MOD.head()

,RGN,SEASON,VHCL_MKCNT,LOAD_DT,LOAD_DT
0,수도권,2차계절제_시작,539748,20230810,20230901
1,수도권,2차계절제_종료,515615,20230810,20230901
2,수도권,3차계절제_시작,459110,20230810,20230901
3,수도권,3차계절제_종료,436921,20230810,20230901
4,수도권,4차계절제_시작,394533,20230810,20230901


In [187]:
STD_BD_GRD5_LEM_N_MOD.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   RGN         12 non-null     object
 1   SEASON      12 non-null     object
 2   VHCL_MKCNT  12 non-null     int64 
 3   LOAD_DT     12 non-null     int64 
 4   LOAD_DT     12 non-null     object
dtypes: int64(2), object(3)
memory usage: 608.0+ bytes


### [출력] [D] STD_BD_GRD5_LEM_N_MOD

In [188]:
# expdf = STD_BD_GRD5_LEM_N_MOD
# table_nm = 'STD_BD_GRD5_LEM_N_MOD'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     # if 'float' in expdf[column].dtype.name:
#     #     sql += column + ' float'
#     # elif 'int' in expdf[column].dtype.name:
#     #     sql += column + ' number'
#     # else:
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 5s
# we.import_from_pandas(expdf, table_nm)

In [189]:
# # 1.3s
# STD_BD_GRD5_LEM_N_MOD.to_csv(os.path.join(df3_fold, 'STD_BD_GRD5_LEM_N_MOD.csv'), index=False)

## 차대번호별 운행제한 적발 현황

In [190]:
# DNSTY_STDR_ID(농도기준아이디) : 실발령(C011), 모의발령(C012)
# TY_STDR_ID(유형기준아이디) : 비상시(T001), 계절제(T002)

In [191]:
is_total.head()

,적발번호,발령번호,차대번호,등록시도코드,등록시군구코드,적발지역코드,단속일,유형기준아이디,농도기준아이디,등록시도,등록시군구,적발시도코드,적발시군구코드,적발시도,적발시군구
0,51565757,1821,KMFPA18TP5C001735,41,170,28000,NaN,T001,C012,경기도,안양시,28,0,인천광역시,NaN
1,51565758,1821,KNAJE55135K039713,28,260,28000,NaN,T001,C012,인천광역시,서구,28,0,인천광역시,NaN
2,51652557,1821,KL5UL42PD7U000105,11,650,11000,NaN,T001,C012,서울특별시,서초구,11,0,서울특별시,NaN
3,51652540,1821,KMFGA17PP6C041875,41,460,11000,NaN,T001,C012,경기도,용인시,11,0,서울특별시,NaN
4,51652532,1821,KMFZCS7JP6U175495,41,170,11000,NaN,T001,C012,경기도,안양시,11,0,서울특별시,NaN


In [192]:
is_total.shape

(108436093, 15)

In [193]:
is_total['농도기준아이디'].unique()

array(['C012', 'C011'], dtype=object)

In [194]:
is_total['유형기준아이디'].unique()

array(['T001', 'T002'], dtype=object)

In [195]:
# 11.0s
# DNSTY_STDR_ID(농도기준아이디) : 실발령(C011), 모의발령(C012)
# TY_STDR_ID(유형기준아이디) : 비상시(T001), 계절제(T002)
# 계절제
is_season = is_total.loc[(is_total['농도기준아이디'] == 'C011') & (is_total['유형기준아이디'] == 'T002')].reset_index(drop=True)
is_season.shape

(16382044, 15)

In [196]:
is_season['단속일'].dtype

dtype('float64')

In [197]:
is_season['단속일'].describe()

count     387538.00
mean    20214080.37
std         8988.34
min     20201201.00
25%     20210120.00
50%     20211202.00
75%     20220316.00
max     20230331.00
Name: 단속일, dtype: float64

In [198]:
int(today_date[:4])

2023

In [199]:
# 계절제 1차(2019.12 ~ 2020.3)
# 계절제 2차(2020.12 ~ 2021.3)
# 계절제 3차(2021.12 ~ 2022.3)
# 계절제 4차(2022.12 ~ 2023.3)

for yr in range(2019, int(today_date[:4])):
    start_date = f'{yr}1130'
    end_date = f'{yr+1}0401'
    is_season.loc[(is_season['단속일'] > int(start_date)) & (is_season['단속일'] < int(end_date)), f'계절제_{yr-2018}차여부'] = 'Y'

In [200]:
is_season.head()

,적발번호,발령번호,차대번호,등록시도코드,등록시군구코드,적발지역코드,단속일,유형기준아이디,농도기준아이디,등록시도,등록시군구,적발시도코드,적발시군구코드,적발시도,적발시군구,계절제_1차여부,계절제_2차여부,계절제_3차여부,계절제_4차여부
0,111616383,2430,KL90F4ADD5HZ00068,45,130,28000,NaN,T002,C011,전라북도,군산시,28,0,인천광역시,NaN,NaN,NaN,NaN,NaN
1,111616380,2430,KL3P9CNF37K000737,28,140,28000,NaN,T002,C011,인천광역시,동구,28,0,인천광역시,NaN,NaN,NaN,NaN,NaN
2,111620590,2430,KPBFA2AD14P125977,26,260,26000,NaN,T002,C011,부산광역시,동래구,26,0,부산광역시,NaN,NaN,NaN,NaN,NaN
3,111620591,2430,KMFLA18AP2C034510,48,120,26000,NaN,T002,C011,경상남도,창원시,26,0,부산광역시,NaN,NaN,NaN,NaN,NaN
4,111622099,2430,KMFLA19AP2C031701,48,250,26000,NaN,T002,C011,경상남도,김해시,26,0,부산광역시,NaN,NaN,NaN,NaN,NaN


In [201]:
is_season.loc[( (is_season['적발시도'] == '부산광역시') | (is_season['적발시도'] == '대구광역시') ) & ((is_season['계절제_2차여부'] == 'Y') | (is_season['계절제_3차여부'] == 'Y'))].shape

(0, 19)

In [202]:
is_season.loc[( (is_season['적발시도'] == '부산광역시') | (is_season['적발시도'] == '대구광역시') ) & (is_season['계절제_4차여부'] == 'Y')].shape

(45942, 19)

In [203]:
is_season.shape, len(is_season['차대번호'].unique())

((16382044, 19), 452851)

In [204]:
is_season.drop_duplicates(['차대번호', '적발시도']).shape

(753349, 19)

In [205]:
agg_dict = {x:'count' for x in is_season.columns if '계절제' in x}
agg_dict

{'계절제_1차여부': 'count',
 '계절제_2차여부': 'count',
 '계절제_3차여부': 'count',
 '계절제_4차여부': 'count'}

In [206]:
limit_season_rename_dict = {x:x.replace('여부','') for x in agg_dict.keys()}
limit_season_rename_dict

{'계절제_1차여부': '계절제_1차',
 '계절제_2차여부': '계절제_2차',
 '계절제_3차여부': '계절제_3차',
 '계절제_4차여부': '계절제_4차'}

In [207]:
limit_season = is_season.groupby(['차대번호'], as_index=False).agg(agg_dict)
limit_season = limit_season.rename(columns=limit_season_rename_dict)
limit_season

,차대번호,계절제_1차,계절제_2차,계절제_3차,계절제_4차
0,000000JN44C000673,0,0,9,0
1,01451201380,0,0,0,0
2,01741320297,0,0,0,0
3,01751300580,0,1,0,0
4,01761300921,0,0,0,0
...,...,...,...,...,...
452846,YV2JSO2C768870642,0,0,1,2
452847,YV2JSO2C76A625817,0,0,0,0
452848,YV2JSO2C88A655606,0,0,0,0
452849,YV2JSO2C968870643,0,0,31,0


In [208]:
limit_season[(limit_season['계절제_1차'] > 0) |(limit_season['계절제_2차'] > 0) |(limit_season['계절제_3차'] > 0) | (limit_season['계절제_4차'] > 0)].shape

(99791, 5)

In [209]:
# 11.0s
# DNSTY_STDR_ID(농도기준아이디) : 실발령(C011), 모의발령(C012)
# TY_STDR_ID(유형기준아이디) : 비상시(T001), 계절제(T002)
# 비상시
is_high = is_total.loc[(is_total['농도기준아이디'] == 'C011') & (is_total['유형기준아이디'] == 'T001')].reset_index(drop=True)
is_high.shape

(753383, 15)

In [210]:
is_high.head()

,적발번호,발령번호,차대번호,등록시도코드,등록시군구코드,적발지역코드,단속일,유형기준아이디,농도기준아이디,등록시도,등록시군구,적발시도코드,적발시군구코드,적발시도,적발시군구
0,114846051,2672,KL4V3TDF13K000505,26,470,31000,NaN,T001,C011,부산광역시,연제구,31,0,울산광역시,NaN
1,114838737,2672,KMFWVH7JP7U801845,26,170,26000,NaN,T001,C011,부산광역시,동구,26,0,부산광역시,NaN
2,114844155,2672,KMJWWH7JP6U744190,31,710,31000,NaN,T001,C011,울산광역시,울주군,31,0,울산광역시,NaN
3,114844158,2672,KMFLA18EP6C012515,31,140,31000,NaN,T001,C011,울산광역시,남구,31,0,울산광역시,NaN
4,114844159,2672,KMFGA17PP7C061877,31,140,31000,NaN,T001,C011,울산광역시,남구,31,0,울산광역시,NaN


In [211]:
limit_high = is_high.groupby(['차대번호'], as_index=False).agg({'단속일':'count'})
limit_high = limit_high.rename(columns={'단속일':'비상시'})
limit_high

,차대번호,비상시
0,000000JN44C000673,1
1,01441201316,0
2,01761301128,1
3,01794200041,0
4,01841300288,1
...,...,...
227030,YV2JS02CX8A655591,0
227031,YV2JSO2C768870642,1
227032,YV2JSO2C76A625817,1
227033,YV2JSO2C88A655606,0


In [212]:
us_total.shape

(3151659, 15)

In [213]:
us_total['적발건수'].dtype

dtype('float64')

In [214]:
limit_alw = us_total.groupby('차대번호', as_index=False).agg({'적발건수':'sum'})
limit_alw = limit_alw.rename(columns={'적발건수':'상시'})
limit_alw

,차대번호,상시
0,01771226923,3.00
1,01781201391,1.00
2,01781212746,1.00
3,01791201714,2.00
4,01792116793,7.00
...,...,...
86533,YV2JSO2C668870647,64.00
86534,YV2JSO2C768870642,4.00
86535,YV2JSO2C76A620052,1.00
86536,YV2JSO2C968870643,86.00


In [215]:
limit_season.shape, limit_high.shape, limit_alw.shape

((452851, 5), (227035, 2), (86538, 2))

In [216]:
limit_sh = limit_season.merge(limit_high, on='차대번호', how='left')
limit_sh.shape

(452851, 6)

In [217]:
limit = limit_sh.merge(limit_alw, on='차대번호', how='left')
limit.shape

(452851, 7)

In [218]:
limit.columns

Index(['차대번호', '계절제_1차', '계절제_2차', '계절제_3차', '계절제_4차', '비상시', '상시'], dtype='object')

In [219]:
limit.iloc[:, 1:].isnull().sum()

계절제_1차         0
계절제_2차         0
계절제_3차         0
계절제_4차         0
비상시       289859
상시        403805
dtype: int64

In [220]:
limit.iloc[:, 1:] = limit.iloc[:, 1:].fillna(0)
limit.iloc[:, 1:].isnull().sum()

계절제_1차    0
계절제_2차    0
계절제_3차    0
계절제_4차    0
비상시       0
상시        0
dtype: int64

In [221]:
limit[['비상시', '상시']].dtypes

비상시    float64
상시     float64
dtype: object

In [222]:
limit['비상시'] = limit['비상시'].astype('int')
limit['상시'] = limit['상시'].astype('int')

In [223]:
limit.head()

,차대번호,계절제_1차,계절제_2차,계절제_3차,계절제_4차,비상시,상시
0,000000JN44C000673,0,0,9,0,1,0
1,01451201380,0,0,0,0,0,0
2,01741320297,0,0,0,0,0,0
3,01751300580,0,1,0,0,0,0
4,01761300921,0,0,0,0,0,0


In [224]:
limit.shape

(452851, 7)

In [225]:
df.shape

(973987, 20)

In [226]:
lmt1 = df.merge(limit, on='차대번호', how='left')
lmt1.shape

(973987, 26)

In [227]:
lmt1.columns

Index(['차대번호', '법정동코드', '차량말소YN', '제원관리번호', '차량연식', '차종', '용도', '최초등록일자',
       '제작일자', '차량관리번호', '연료', '차종유형', 'DPF_YN', '시도코드', '시군구코드', '시도', '시군구',
       '조기폐차상태코드', '조기폐차최종승인YN', '말소일자', '계절제_1차', '계절제_2차', '계절제_3차',
       '계절제_4차', '비상시', '상시'],
      dtype='object')

In [228]:
lmt1.head()

,차대번호,법정동코드,차량말소YN,제원관리번호,차량연식,차종,용도,최초등록일자,제작일자,차량관리번호,...,시군구,조기폐차상태코드,조기폐차최종승인YN,말소일자,계절제_1차,계절제_2차,계절제_3차,계절제_4차,비상시,상시
0,KL3C3AHF27K000073,4785000000,N,A0310007202023307,2007,화물,개인용,20230613.00,20070627.00,KL3C3AHF27K000073-01,...,칠곡군,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,KMFGA17PP7C071816,4427000000,N,00034427000023223,2007,화물,개인용,20230221.00,20061231.00,KMFGA17PP7C071816-01,...,당진시,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,KL3C3AHF27K000082,4481000000,N,A0310007202023307,2007,화물,개인용,20220711.00,20070627.00,KL3C3AHF27K000082-01,...,예산군,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,KRGLAAC3K3B019534,2714000000,N,00034427000033222,2003,화물,개인용,20220414.00,20021231.00,KRGLAAC3K3B019534-01,...,동구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,KL3C3AHF27K000068,4684000000,N,A0310007202023307,2007,화물,개인용,20220413.00,20070618.00,KL3C3AHF27K000068-01,...,무안군,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [229]:
lmt1['시도'].unique()

array(['경상북도', '충청남도', '대구광역시', '전라남도', '경기도', '울산광역시', '부산광역시', '광주광역시',
       '대전광역시', '경상남도', '강원특별자치도', '충청북도', '전라북도', '서울특별시', '인천광역시',
       '제주특별자치도', '세종특별자치시'], dtype=object)

In [230]:
lmt1.loc[(lmt1['시도'] == '서울특별시') | (lmt1['시도'] == '경기도') | (lmt1['시도'] == '인천광역시'), '지역'] = '수도권'
lmt1['지역'] = lmt1['지역'].fillna('수도권외')

In [231]:
lmt1['지역'].unique()

array(['수도권외', '수도권'], dtype=object)

In [232]:
lmt1['DPF_YN'].value_counts(dropna=False)

DPF_YN
NaN    692500
유      281487
Name: count, dtype: int64

In [233]:
lmt1['DPF_YN'] = lmt1['DPF_YN'].fillna('무')

In [234]:
season_start_date = datetime(2020, 12, 1)
season_end_date = datetime(2021, 3, 31)
season_start_date, season_end_date

(datetime.datetime(2020, 12, 1, 0, 0), datetime.datetime(2021, 3, 31, 0, 0))

In [235]:
days = (season_end_date - season_start_date).days
days

120

In [236]:
for one in [x for x in limit_season_rename_dict.values()]:
    lmt1[one + '_일평균'] = lmt1[one] / days

In [237]:
lmt1.columns

Index(['차대번호', '법정동코드', '차량말소YN', '제원관리번호', '차량연식', '차종', '용도', '최초등록일자',
       '제작일자', '차량관리번호', '연료', '차종유형', 'DPF_YN', '시도코드', '시군구코드', '시도', '시군구',
       '조기폐차상태코드', '조기폐차최종승인YN', '말소일자', '계절제_1차', '계절제_2차', '계절제_3차',
       '계절제_4차', '비상시', '상시', '지역', '계절제_1차_일평균', '계절제_2차_일평균', '계절제_3차_일평균',
       '계절제_4차_일평균'],
      dtype='object')

In [238]:
lmt1[[x for x in limit_season_rename_dict.values()] + [x + '_일평균' for x in limit_season_rename_dict.values()]].isnull().sum()

계절제_1차        688161
계절제_2차        688161
계절제_3차        688161
계절제_4차        688161
계절제_1차_일평균    688161
계절제_2차_일평균    688161
계절제_3차_일평균    688161
계절제_4차_일평균    688161
dtype: int64

In [239]:
lmt1[[x for x in limit_season_rename_dict.values()] + [x + '_일평균' for x in limit_season_rename_dict.values()]] = lmt1[[x for x in limit_season_rename_dict.values()] + [x + '_일평균' for x in limit_season_rename_dict.values()]].fillna(0)
lmt1[[x for x in limit_season_rename_dict.values()] + [x + '_일평균' for x in limit_season_rename_dict.values()]].isnull().sum()

계절제_1차        0
계절제_2차        0
계절제_3차        0
계절제_4차        0
계절제_1차_일평균    0
계절제_2차_일평균    0
계절제_3차_일평균    0
계절제_4차_일평균    0
dtype: int64

### 수정 코드(2023.08.31)

In [240]:
is_season.columns

Index(['적발번호', '발령번호', '차대번호', '등록시도코드', '등록시군구코드', '적발지역코드', '단속일', '유형기준아이디',
       '농도기준아이디', '등록시도', '등록시군구', '적발시도코드', '적발시군구코드', '적발시도', '적발시군구',
       '계절제_1차여부', '계절제_2차여부', '계절제_3차여부', '계절제_4차여부'],
      dtype='object')

In [241]:
is_season['계절제_2차여부'].unique()

array([nan, 'Y'], dtype=object)

In [242]:
is_season.shape, len(is_season['차대번호'].unique())

((16382044, 19), 452851)

In [243]:
ss_df = is_season.merge(df, on='차대번호', how='left')
ss_df.shape

(16382044, 38)

In [244]:
ss_df[((ss_df['계절제_2차여부'] == 'Y') | ((ss_df['계절제_3차여부'] == 'Y'))) & ((ss_df['적발시도'] == '부산광역시') | (ss_df['적발시도'] == '대구광역시'))].shape

(0, 38)

In [245]:
ss_df.columns

Index(['적발번호', '발령번호', '차대번호', '등록시도코드', '등록시군구코드', '적발지역코드', '단속일', '유형기준아이디',
       '농도기준아이디', '등록시도', '등록시군구', '적발시도코드', '적발시군구코드', '적발시도', '적발시군구',
       '계절제_1차여부', '계절제_2차여부', '계절제_3차여부', '계절제_4차여부', '법정동코드', '차량말소YN',
       '제원관리번호', '차량연식', '차종', '용도', '최초등록일자', '제작일자', '차량관리번호', '연료', '차종유형',
       'DPF_YN', '시도코드', '시군구코드', '시도', '시군구', '조기폐차상태코드', '조기폐차최종승인YN',
       '말소일자'],
      dtype='object')

In [246]:
ss_df['적발시도'].unique()

array(['인천광역시', '부산광역시', '대구광역시', '서울특별시', '경기도'], dtype=object)

In [247]:
ss_df.loc[(ss_df['적발시도'] == '서울특별시') | (ss_df['적발시도'] == '경기도') | (ss_df['적발시도'] == '인천광역시'), '적발지역'] = '수도권'
ss_df.loc[(ss_df['적발시도'] == '부산광역시') | (ss_df['적발시도'] == '대구광역시'), '적발지역'] = '수도권외'

In [248]:
ss_df['적발지역'].unique()

array(['수도권', '수도권외'], dtype=object)

In [249]:
ss_df['적발지역'].value_counts(dropna=False)

적발지역
수도권     15429850
수도권외      952194
Name: count, dtype: int64

In [312]:
ss_df['DPF_YN'].value_counts(dropna=False)

DPF_YN
유      9791226
NaN    6590818
Name: count, dtype: int64

In [313]:
ss_df['DPF_YN'] = ss_df['DPF_YN'].fillna('무')
ss_df['DPF_YN'].value_counts(dropna=False)

DPF_YN
유    9791226
무    6590818
Name: count, dtype: int64

In [314]:
ss_df['테이블생성일자'] = today_date

In [315]:
[x for x in limit_season_rename_dict.keys()]

['계절제_1차여부', '계절제_2차여부', '계절제_3차여부', '계절제_4차여부']

In [316]:
season_col = ['테이블생성일자', '차대번호'] + ['적발지역', '적발시도', 'DPF_YN', '차종', '차종유형'] + [x for x in limit_season_rename_dict.keys()]

In [317]:
season = ss_df[season_col]
season.shape

(16382044, 11)

In [318]:
season['적발시도'].unique()

array(['인천광역시', '부산광역시', '대구광역시', '서울특별시', '경기도'], dtype=object)

In [319]:
season[(season['계절제_1차여부'] == 'Y') & (season['적발시도'] == '부산광역시')].shape, season[(season['계절제_1차여부'] == 'ㅛ') & (season['적발시도'] == '대구광역시')].shape

((0, 11), (0, 11))

In [320]:
season[(season['계절제_2차여부'] == 'Y') & (season['적발시도'] == '부산광역시')].shape, season[(season['계절제_2차여부'] == 'Y') & (season['적발시도'] == '대구광역시')].shape

((0, 11), (0, 11))

In [321]:
season[(season['계절제_3차여부'] == 'Y') & (season['적발시도'] == '부산광역시')].shape, season[(season['계절제_3차여부'] == 'Y') & (season['적발시도'] == '대구광역시')].shape

((0, 11), (0, 11))

In [302]:
limit_season.shape

(452851, 5)

In [324]:
is_season['단속일'].isnull().sum()

15994506

In [325]:
ss_df.columns

Index(['적발번호', '발령번호', '차대번호', '등록시도코드', '등록시군구코드', '적발지역코드', '단속일', '유형기준아이디',
       '농도기준아이디', '등록시도', '등록시군구', '적발시도코드', '적발시군구코드', '적발시도', '적발시군구',
       '계절제_1차여부', '계절제_2차여부', '계절제_3차여부', '계절제_4차여부', '법정동코드', '차량말소YN',
       '제원관리번호', '차량연식', '차종', '용도', '최초등록일자', '제작일자', '차량관리번호', '연료', '차종유형',
       'DPF_YN', '시도코드', '시군구코드', '시도', '시군구', '조기폐차상태코드', '조기폐차최종승인YN',
       '말소일자', '적발지역', '테이블생성일자'],
      dtype='object')

In [326]:
ss_df[(ss_df['단속일'].isnull() == True) & ((ss_df['계절제_1차여부']) | (ss_df['계절제_1차여부']) | (ss_df['계절제_1차여부']))].shape

(0, 40)

In [305]:
ss_df.drop_duplicates(['등록시도', '차대번호']).shape, ss_df.drop_duplicates('차대번호').shape

((473851, 40), (452851, 40))

In [307]:
temp = ss_df.drop_duplicates(['등록시도', '차대번호'])

(21000, 40)

In [308]:
temp.loc[temp['차대번호'].duplicated()].shape

(21000, 40)

In [323]:
temp.loc[temp['차대번호'].duplicated(keep=False)].sort_values('차대번호').head(10)

,적발번호,발령번호,차대번호,등록시도코드,등록시군구코드,적발지역코드,단속일,유형기준아이디,농도기준아이디,등록시도,...,DPF_YN,시도코드,시군구코드,시도,시군구,조기폐차상태코드,조기폐차최종승인YN,말소일자,적발지역,테이블생성일자
13618976,12397135,498,01761301128,41,570,28000,NaN,T002,C011,경기도,...,NaN,28.00,710.00,인천광역시,강화군,NaN,NaN,NaN,수도권,20230901
10653830,41407527,1384,01761301128,28,710,41570,20220217.00,T002,C011,인천광역시,...,NaN,28.00,710.00,인천광역시,강화군,NaN,NaN,NaN,수도권,20230901
14772763,107226544,2416,02831187228EOQN,30,230,27000,NaN,T002,C011,대전광역시,...,NaN,30.00,230.00,대전광역시,대덕구,NaN,NaN,NaN,수도권외,20230901
7426924,6014535,448,02831187228EOQN,28,110,41170,NaN,T002,C011,인천광역시,...,NaN,30.00,230.00,대전광역시,대덕구,NaN,NaN,NaN,수도권,20230901
5852189,3906777,430,03824219227,41,280,41280,NaN,T002,C011,경기도,...,NaN,46.00,110.00,전라남도,목포시,NaN,NaN,NaN,수도권,20230901
16343469,136904671,2500,03824219227,46,110,11000,20230331.00,T002,C011,전라남도,...,NaN,46.00,110.00,전라남도,목포시,NaN,NaN,NaN,수도권,20230901
5929219,3993320,431,110244230031,28,177,28000,NaN,T002,C011,인천광역시,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,수도권,20230901
1287140,119655671,2455,110244230031,11,110,28000,NaN,T002,C011,서울특별시,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,수도권,20230901
2620530,128638343,2479,118311301989,50,110,27000,NaN,T002,C011,제주특별자치도,...,NaN,50.00,110.00,제주특별자치도,제주시,NaN,NaN,NaN,수도권외,20230901
6163912,4287875,434,118311301989,11,680,41130,NaN,T002,C011,서울특별시,...,NaN,50.00,110.00,제주특별자치도,제주시,NaN,NaN,NaN,수도권,20230901


In [258]:
cdict = {
    '테이블생성일자':'LOAD_DT', 
    '차대번호':'VIN', 
    '적발지역':'DSCL_RGN', 
    '적발시도':'DSCL_CTPV', 
    'DPF_YN':'DPF_EXTRNS_YN', 
    '차종':'VHCTY_CD', 
    '차종유형':'VHCTY_TY', 
}
for one in limit_season_rename_dict.keys():
    cdict[one] = one.replace('계절제', 'SEASON').replace('차여부', 'ODR_CRDN_YN')
cdict

{'테이블생성일자': 'LOAD_DT',
 '차대번호': 'VIN',
 '적발지역': 'DSCL_RGN',
 '적발시도': 'DSCL_CTPV',
 'DPF_YN': 'DPF_EXTRNS_YN',
 '차종': 'VHCTY_CD',
 '차종유형': 'VHCTY_TY',
 '계절제_1차여부': 'SEASON_1ODR_CRDN_YN',
 '계절제_2차여부': 'SEASON_2ODR_CRDN_YN',
 '계절제_3차여부': 'SEASON_3ODR_CRDN_YN',
 '계절제_4차여부': 'SEASON_4ODR_CRDN_YN'}

In [259]:
# season = lmt1[season_col]

In [260]:
# cdict = {
#     '테이블생성일자':'LOAD_DT', 
#     '차대번호':'VIN', 
#     '적발지역':'RGN', 
#     '적발시도':'CTPV', 
#     'DPF_YN':'DPF_EXTRNS_YN', 
#     '차종':'VHCTY_CD', 
#     '차종유형':'VHCTY_TY', 
# }
# for one in limit_season_rename_dict.values():
#     cdict[one] = one.replace('계절제', 'SEASON').replace('차', 'ODR_CRDN_NOCS')
# cdict

In [261]:
STD_BD_SEASON_CRDN_NOCS_CURSTT = season.rename(columns=cdict)
STD_BD_SEASON_CRDN_NOCS_CURSTT.columns

Index(['LOAD_DT', 'VIN', 'DSCL_RGN', 'DSCL_CTPV', 'DPF_EXTRNS_YN', 'VHCTY_CD',
       'VHCTY_TY', 'SEASON_1ODR_CRDN_YN', 'SEASON_2ODR_CRDN_YN',
       'SEASON_3ODR_CRDN_YN', 'SEASON_4ODR_CRDN_YN'],
      dtype='object')

In [262]:
list(STD_BD_SEASON_CRDN_NOCS_CURSTT.columns)

['LOAD_DT',
 'VIN',
 'DSCL_RGN',
 'DSCL_CTPV',
 'DPF_EXTRNS_YN',
 'VHCTY_CD',
 'VHCTY_TY',
 'SEASON_1ODR_CRDN_YN',
 'SEASON_2ODR_CRDN_YN',
 'SEASON_3ODR_CRDN_YN',
 'SEASON_4ODR_CRDN_YN']

In [263]:
STD_BD_SEASON_CRDN_NOCS_CURSTT.shape

(16382044, 11)

In [264]:
STD_BD_SEASON_CRDN_NOCS_CURSTT.head()

,LOAD_DT,VIN,DSCL_RGN,DSCL_CTPV,DPF_EXTRNS_YN,VHCTY_CD,VHCTY_TY,SEASON_1ODR_CRDN_YN,SEASON_2ODR_CRDN_YN,SEASON_3ODR_CRDN_YN,SEASON_4ODR_CRDN_YN
0,20230901,KL90F4ADD5HZ00068,수도권,인천광역시,유,승용,대형,NaN,NaN,NaN,NaN
1,20230901,KL3P9CNF37K000737,수도권,인천광역시,유,화물,대형,NaN,NaN,NaN,NaN
2,20230901,KPBFA2AD14P125977,수도권외,부산광역시,NaN,승용,대형,NaN,NaN,NaN,NaN
3,20230901,KMFLA18AP2C034510,수도권외,부산광역시,유,화물,대형,NaN,NaN,NaN,NaN
4,20230901,KMFLA19AP2C031701,수도권외,부산광역시,NaN,화물,대형,NaN,NaN,NaN,NaN


### [출력] STD_BD_SEASON_CRDN_NOCS_CURSTT

In [265]:
# expdf = STD_BD_SEASON_CRDN_NOCS_CURSTT
# table_nm = 'STD_BD_SEASON_CRDN_NOCS_CURSTT'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 5s
# we.import_from_pandas(expdf, table_nm)

In [266]:
# # 
# STD_BD_SEASON_CRDN_NOCS_CURSTT.to_csv(os.path.join(df3_fold, 'STD_BD_SEASON_CRDN_NOCS_CURSTT.csv'), index=False)

## 계절제별 적발건수

In [267]:
limit_season_rename_dict

{'계절제_1차여부': '계절제_1차',
 '계절제_2차여부': '계절제_2차',
 '계절제_3차여부': '계절제_3차',
 '계절제_4차여부': '계절제_4차'}

In [268]:
season_tot = lmt1[[x + '_일평균' for x in limit_season_rename_dict.values()]].sum().reset_index()
season_tot = season_tot.rename(columns={'index':'계절제차수', 0:'일평균적발건수'})
season_tot['계절제차수'] = season_tot['계절제차수'].str.replace('계절제_', '').str.replace('_일평균', '')
season_tot

,계절제차수,일평균적발건수
0,1차,0.00
1,2차,684.51
2,3차,508.93
3,4차,556.02


In [269]:
season_tot['테이블생성일자'] = today_date

In [270]:
season_tot.columns

Index(['계절제차수', '일평균적발건수', '테이블생성일자'], dtype='object')

In [271]:
cdict = {
    '계절제차수':'SEASON_ORD', 
    '일평균적발건수':'DY_AVRG_CRDN_NOCS', 
    '테이블생성일자':'LOAD_DT', 
    }
STD_BD_SEASON_DY_AVRG_CRDN_NOCS = season_tot.rename(columns=cdict)
STD_BD_SEASON_DY_AVRG_CRDN_NOCS.columns

Index(['SEASON_ORD', 'DY_AVRG_CRDN_NOCS', 'LOAD_DT'], dtype='object')

In [272]:
list(STD_BD_SEASON_DY_AVRG_CRDN_NOCS.columns)

['SEASON_ORD', 'DY_AVRG_CRDN_NOCS', 'LOAD_DT']

### [출력] STD_BD_SEASON_DY_AVRG_CRDN_NOCS

In [273]:
# expdf = STD_BD_SEASON_DY_AVRG_CRDN_NOCS
# table_nm = 'STD_BD_SEASON_DY_AVRG_CRDN_NOCS'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     # if 'float' in expdf[column].dtype.name:
#     #     sql += column + ' float'
#     # elif 'int' in expdf[column].dtype.name:
#     #     sql += column + ' number'
#     # else:
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 5s
# we.import_from_pandas(expdf, table_nm)

In [274]:
# # 
# STD_BD_SEASON_DY_AVRG_CRDN_NOCS.to_csv(os.path.join(df3_fold, 'STD_BD_SEASON_DY_AVRG_CRDN_NOCS.csv'), index=False)

## 지역별 상시운행제한 단속 현황

In [275]:
us_total.shape

(3151659, 15)

In [276]:
us_total2 = us_total.merge(limit_alw, on='차대번호', how='left')
us_total2.shape

(3151659, 16)

In [277]:
us_total2.head()

,번호,차대번호,적발유형,단속지역코드,등록시도코드,등록시군구코드,적발건수,적발년월,법정동코드,등록시도,등록시군구,단속시도코드,단속시군구코드,단속시도,단속시군구,상시
0,8389408,KMFZCD7JP5U081362,S,11000,11,140,NaN,NaN,1114000000,서울특별시,중구,11,0,서울특별시,NaN,8.00
1,8389410,KN3HNP6D15K094646,S,11000,11,140,NaN,NaN,1114000000,서울특별시,중구,11,0,서울특별시,NaN,74.00
2,8389713,KNAMA75432S302063,S,11000,11,140,1.00,201908.00,1114000000,서울특별시,중구,11,0,서울특별시,NaN,36.00
3,8389714,KMFLA19RPYC015091,S,11000,11,140,1.00,201908.00,1114000000,서울특별시,중구,11,0,서울특별시,NaN,68.00
4,8389716,KMHWP81JP5U664886,S,11000,11,140,NaN,NaN,1114000000,서울특별시,중구,11,0,서울특별시,NaN,37.00


In [211]:
# us_total2.loc[(us_total2['시도'] == '서울특별시') | (us_total2['시도'] == '경기도') | (us_total2['시도'] == '인천광역시'), '지역'] = '수도권'
# us_total2['지역'] = us_total2['지역'].fillna('수도권외')

In [278]:
us_total2['단속시도'].unique()

array(['서울특별시', '경기도', '인천광역시'], dtype=object)

In [279]:
us_total2.loc[(us_total2['단속시도'] == '서울특별시') | (us_total2['단속시도'] == '경기도') | (us_total2['단속시도'] == '인천광역시'), '단속지역'] = '수도권'

In [281]:
us_total2['적발년월'] = us_total2['적발년월'].astype('str')
us_total2['적발년월_년'] = us_total2['적발년월'].str[:4]

In [286]:
# 4s
us_total2 = us_total2.sort_values('적발년월_년', ascending=True).drop_duplicates('차대번호').reset_index(drop=True)
us_total2.shape

(86538, 18)

In [287]:
us_total2 = us_total2.drop(['적발건수'], axis=1)

In [288]:
us_total2 = us_total2.rename(columns={'적발년월_년':'적발년도', '상시':'적발건수'})

In [289]:
us_total2['적발건수'].dtype

dtype('float64')

In [298]:
us_total2['단속시도'].isnull().sum()

0

In [299]:
us_total2['단속지역'].isnull().sum()

0

In [290]:
orditm = us_total2.loc[(us_total2['적발건수'] > 0) & (us_total2['적발년도'].isnull() == False), [
    '차대번호',
    '적발년도',
    '적발건수',
    '단속지역',
    '단속시도',
]]

In [291]:
orditm.shape

(84293, 5)

In [292]:
orditm['적발건수'].describe()

count   84293.00
mean       23.94
std        45.81
min         1.00
25%         2.00
50%         8.00
75%        27.00
max      1022.00
Name: 적발건수, dtype: float64

In [293]:
orditm['테이블생성일자'] = today_date

In [296]:
cdict = {
    '테이블생성일자':'LOAD_DT', 
    '차대번호':'VIN', 
    '적발년도':'DSCL_YR', 
    '적발건수':'DSCL_NOCS', 
    '단속지역':'RGN',
    '단속시도':'CTPV', 
}

In [297]:
STD_BD_ORDITM_DSCL_CURSTT = orditm.rename(columns=cdict)
STD_BD_ORDITM_DSCL_CURSTT.columns

Index(['VIN', 'DSCL_YR', 'DSCL_NOCS', 'RGN', 'CTPV', 'LOAD_DT'], dtype='object')

### [출력] STD_BD_ORDITM_DSCL_CURSTT

In [223]:
# expdf = STD_BD_ORDITM_DSCL_CURSTT
# table_nm = 'STD_BD_ORDITM_DSCL_CURSTT'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     # if 'float' in expdf[column].dtype.name:
#     #     sql += column + ' float'
#     # elif 'int' in expdf[column].dtype.name:
#     #     sql += column + ' number'
#     # else:
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 5s
# we.import_from_pandas(expdf, table_nm)

In [224]:
# # 1.3s
# STD_BD_ORDITM_DSCL_CURSTT.to_csv(os.path.join(df3_fold, 'STD_BD_ORDITM_DSCL_CURSTT.csv'), index=False)

## 적발지역별 계절제 단속 현황 출력

In [225]:
is_season.shape

(13012643, 17)

In [226]:
is_season.head()

,적발번호,발령번호,등록시도코드,등록시군구코드,적발지역코드,차대번호,단속일,농도기준아이디,유형기준아이디,등록시도,등록시군구,적발시도,적발시군구,계절제_1차여부,계절제_2차여부,계절제_3차여부,계절제_4차여부
0,112681813,2433,27,290,41360,KMFDA18TP6C008119,NaN,C011,T002,대구광역시,달서구,경기도,남양주시,NaN,NaN,NaN,NaN
1,112681514,2433,41,630,11000,KMCVK17PP5C018376,NaN,C011,T002,경기도,양주시,서울특별시,NaN,NaN,NaN,NaN,NaN
2,112680623,2433,27,170,27000,KNAJE55136K210778,NaN,C011,T002,대구광역시,서구,대구광역시,NaN,NaN,NaN,NaN,NaN
3,112680712,2433,41,190,28000,KMFXKS7BPYU391175,NaN,C011,T002,경기도,부천시,인천광역시,NaN,NaN,NaN,NaN,NaN
4,112681720,2433,48,250,26000,XLEP6X20024473786,NaN,C011,T002,경상남도,김해시,부산광역시,NaN,NaN,NaN,NaN,NaN


In [227]:
is_season2 = is_season.sort_values(['적발시도', '적발시군구']).drop_duplicates(['차대번호', '적발시도']).reset_index(drop=True)
is_season2.head()

,적발번호,발령번호,등록시도코드,등록시군구코드,적발지역코드,차대번호,단속일,농도기준아이디,유형기준아이디,등록시도,등록시군구,적발시도,적발시군구,계절제_1차여부,계절제_2차여부,계절제_3차여부,계절제_4차여부
0,112681849,2433,41,360,41820,KMFLA19AP1C022857,NaN,C011,T002,경기도,남양주시,경기도,가평군,NaN,NaN,NaN,NaN
1,112683561,2433,28,177,41820,KL3K9ARF15K000039,NaN,C011,T002,인천광역시,미추홀구,경기도,가평군,NaN,NaN,NaN,NaN
2,112681848,2433,11,290,41820,KMHSJ81WP6U035856,NaN,C011,T002,서울특별시,성북구,경기도,가평군,NaN,NaN,NaN,NaN
3,112681850,2433,41,820,41820,KMCVK17LP4C197621,NaN,C011,T002,경기도,가평군,경기도,가평군,NaN,NaN,NaN,NaN
4,112685405,2433,41,360,41820,KMFGA17PP7C069955,NaN,C011,T002,경기도,남양주시,경기도,가평군,NaN,NaN,NaN,NaN


In [228]:
is_season2.shape

(1125001, 17)

In [229]:
is_lmt = is_season2.merge(limit_season, on='차대번호', how='left')
is_lmt.shape

(1125001, 21)

In [230]:
is_lmt.loc[(is_lmt['적발시도'] == '서울특별시') | (is_lmt['적발시도'] == '경기도') | (is_lmt['적발시도'] == '인천광역시'), '적발지역'] = '수도권'
is_lmt['적발지역'] = is_lmt['적발지역'].fillna('수도권외')
# is_lmt.loc[is_lmt['등록시도'] == '강원도', '등록시도'] = '강원특별자치도'

In [ ]:
is_lmt['적발시도코드'] = is_lmt['적발지역코드'].str[:2]
is_lmt['적발시도코드'].head()

In [231]:
is_lmt.columns

Index(['적발번호', '발령번호', '등록시도코드', '등록시군구코드', '적발지역코드', '차대번호', '단속일', '농도기준아이디',
       '유형기준아이디', '등록시도', '등록시군구', '적발시도', '적발시군구', '계절제_1차여부', '계절제_2차여부',
       '계절제_3차여부', '계절제_4차여부', '계절제_1차', '계절제_2차', '계절제_3차', '계절제_4차', '적발지역'],
      dtype='object')

In [232]:
is_lmt2 = is_lmt[[
    '적발지역',
    '적발시도', 
    '적발시도코드', # !!! 수정
    '등록시도',  # !!! 수정
    '차대번호', 
    '계절제_1차', 
    '계절제_2차', 
    '계절제_3차', 
    '계절제_4차', 
]]

In [233]:
is_lmt2.shape

(1125001, 7)

In [234]:
df.shape, len(df['차대번호'].unique())

((1162311, 20), 1162311)

In [235]:
df.columns

Index(['차대번호', '법정동코드', '차량말소YN', '제원관리번호', '차량연식', '차종', '용도', '최초등록일자',
       '제작일자', '차량관리번호', '연료', '차종유형', 'DPF_YN', '시도코드', '시군구코드', '시도', '시군구',
       '조기폐차상태코드', '조기폐차최종승인YN', '말소일자'],
      dtype='object')

In [236]:
dfm = df.sort_values('최초등록일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
dfm.shape

(1162311, 20)

In [237]:
slimit = is_lmt2.merge(dfm[['차대번호', '차종', '차종유형']], on='차대번호', how='left')

In [238]:
slimit.shape

(1125001, 9)

In [239]:
slimit.head()

,적발지역,적발시도,차대번호,계절제_1차,계절제_2차,계절제_3차,계절제_4차,차종,차종유형
0,수도권,경기도,KMFLA19AP1C022857,0,0,0,0,화물,중형
1,수도권,경기도,KL3K9ARF15K000039,0,0,0,0,화물,대형
2,수도권,경기도,KMHSJ81WP6U035856,0,0,0,0,승용,대형
3,수도권,경기도,KMCVK17LP4C197621,0,0,0,0,화물,중형
4,수도권,경기도,KMFGA17PP7C069955,0,0,0,0,화물,중형


In [240]:
slimit['적발시도'].unique()

array(['경기도', '대구광역시', '부산광역시', '서울특별시', '인천광역시'], dtype=object)

In [241]:
# 시도명 2글자로 수정
slimit['적발시도'] = slimit['적발시도'].map({'경기도':'경기', '대구광역시':'대구', '부산광역시':'부산', '서울특별시':'서울', '인천광역시':'인천'})
slimit['적발시도'].unique()

array(['경기', '대구', '부산', '서울', '인천'], dtype=object)

In [242]:
# 건수 0 -> nan
slimit.loc[slimit['계절제_1차'] == 0, '계절제_1차'] = np.nan
slimit.loc[slimit['계절제_2차'] == 0, '계절제_2차'] = np.nan
slimit.loc[slimit['계절제_3차'] == 0, '계절제_3차'] = np.nan
slimit.loc[slimit['계절제_4차'] == 0, '계절제_4차'] = np.nan

In [243]:
slimit.head()

,적발지역,적발시도,차대번호,계절제_1차,계절제_2차,계절제_3차,계절제_4차,차종,차종유형
0,수도권,경기,KMFLA19AP1C022857,NaN,NaN,NaN,NaN,화물,중형
1,수도권,경기,KL3K9ARF15K000039,NaN,NaN,NaN,NaN,화물,대형
2,수도권,경기,KMHSJ81WP6U035856,NaN,NaN,NaN,NaN,승용,대형
3,수도권,경기,KMCVK17LP4C197621,NaN,NaN,NaN,NaN,화물,중형
4,수도권,경기,KMFGA17PP7C069955,NaN,NaN,NaN,NaN,화물,중형


In [244]:
slimit.columns

Index(['적발지역', '적발시도', '차대번호', '계절제_1차', '계절제_2차', '계절제_3차', '계절제_4차', '차종',
       '차종유형'],
      dtype='object')

In [245]:
slimit['테이블생성일자'] = today_date

In [246]:
cdict = {
    '테이블생성일자':'LOAD_DT', 
    '적발지역':'DSCL_RGN', 
    '적발시도':'DSCL_CTPV', 
    '적발시도코드':'DSCL_CTPV_CD', # !!! 수정
    '등록시도':'RED_CTPV', # !!! 수정
    '차대번호':'VIN', 
    '계절제_1차':'SEASON_1ODR_CRDN_NOCS', 
    '계절제_2차':'SEASON_2ODR_CRDN_NOCS', 
    '계절제_3차':'SEASON_3ODR_CRDN_NOCS', 
    '계절제_4차':'SEASON_4ODR_CRDN_NOCS', 
    '차종':'VHCTY_CD',
    '차종유형':'VHCTY_TY', 
}

In [247]:
STD_BD_SEASON_DSCL_RGN_CURSTT = slimit.rename(columns=cdict)
STD_BD_SEASON_DSCL_RGN_CURSTT.columns

Index(['DSCL_RGN', 'DSCL_CTPV', 'VIN', 'SEASON_1ODR_CRDN_NOCS',
       'SEASON_2ODR_CRDN_NOCS', 'SEASON_3ODR_CRDN_NOCS',
       'SEASON_4ODR_CRDN_NOCS', 'VHCTY_CD', 'VHCTY_TY', 'LOAD_DT'],
      dtype='object')

### [출력] STD_BD_SEASON_DSCL_RGN_CURSTT

In [248]:
# expdf = STD_BD_SEASON_DSCL_RGN_CURSTT
# table_nm = 'STD_BD_SEASON_DSCL_RGN_CURSTT'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 5s
# we.import_from_pandas(expdf, table_nm)

In [ ]:
# # 1.3s
# STD_BD_SEASON_DSCL_RGN_CURSTT.to_csv(os.path.join(df3_fold, 'STD_BD_SEASON_DSCL_RGN_CURSTT.csv'), index=False)

## 5등급 저감사업

In [330]:
carr.shape

(2378598, 10)

In [331]:
ce = carr.merge(elpm, on='차대번호', how='left')
ce.shape

(2378598, 13)

In [332]:
dfe = ce.merge(attr, on='차대번호', how='left')
dfe.shape

(2378598, 15)

In [333]:
errc.shape

(1190109, 21)

In [334]:
dfe.columns

Index(['차대번호', '법정동코드', '차량말소YN', '제원관리번호', '차량연식', '차종', '용도', '최초등록일자',
       '제작일자', '차량관리번호', '조기폐차상태코드', '조기폐차최종승인YN', '말소일자', '저감장치구분', 'DPF_YN'],
      dtype='object')

In [335]:
errc.columns

Index(['차대번호', '법정동코드', '차량말소YN_x', '제원관리번호', '차량연식', '차종', '용도', '최초등록일자',
       '제작일자', '차량관리번호', '연료', '차종유형', '차량말소YN_y', '변경일자', '시도코드', '시군구코드',
       '시도', '시군구', '변경일자_년', '변경일자_월', '변경일자_일'],
      dtype='object')

In [336]:
dfe['최초등록일자'] = dfe['최초등록일자'].astype('str')
dfe['최초등록일자_년'] = dfe['최초등록일자'].str[:4]
dfe['최초등록일자_월'] = dfe['최초등록일자'].str[4:6]
dfe['최초등록일자_일'] = dfe['최초등록일자'].str[6:8]

In [337]:
dfe['말소일자_년'] = dfe['말소일자'].astype('str').str[:4]
dfe['말소일자_월'] = dfe['말소일자'].astype('str').str[4:6]
dfe['말소일자_일'] = dfe['말소일자'].astype('str').str[6:8]

In [338]:
errc['변경일자'] = errc['변경일자'].astype('str')
errc['변경일자_년'] = errc['변경일자'].str[:4]
errc['변경일자_월'] = errc['변경일자'].str[4:6]
errc['변경일자_일'] = errc['변경일자'].str[6:8]

In [339]:
errc.shape

(1190109, 21)

In [340]:
ere = errc.merge(elpm, on='차대번호', how='left')
ere.shape

(1190109, 24)

In [341]:
erea = ere.merge(attr, on='차대번호', how='left')
erea.shape

(1190109, 26)

In [342]:
# 연도 설정
year = '2022'
# year = today_date[:4]
dfe['연도'] = year

In [343]:
def knd1(x):
    if '1종' in x.unique():
        return x.value_counts()['1종']
    else:
        return 0

In [344]:
def knd2(x):
    if '1종+SCR' in x.unique():
        return x.value_counts()['1종+SCR']
    else:
        return 0

In [345]:
# 2022년 차량 대수
grp1 = dfe[dfe['차량말소YN'] == 'N'].groupby(['연도']).agg({'차대번호':'count', '저감장치구분':[knd1, knd2]}).reset_index()
grp1.columns = ['연도', '차량대수', '저감장치(1종)', '저감장치(1종+SCR)']
grp1

,연도,차량대수,저감장치(1종),저감장치(1종+SCR)
0,2022,1162311,316374,2666


In [346]:
# 연도별 등록대수
grp2 = dfe[dfe['차량말소YN'] == 'N'].groupby(['최초등록일자_년']).agg({'차대번호':'count', '저감장치구분':[knd1, knd2]}).reset_index()
grp2.columns = ['연도', '등록대수', '등록저감장치(1종)', '등록저감장치(1종+SCR)']
grp2

,연도,등록대수,등록저감장치(1종),등록저감장치(1종+SCR)
0,1950,1,0,0
1,1955,1,0,0
2,1958,1,0,0
3,1962,10,0,0
4,1963,2,0,0
...,...,...,...,...
59,2018,61,15,0
60,2019,68,9,1
61,2020,24,6,1
62,2021,9,2,0


In [347]:
# 연도별 말소대수
grp3 = erea.groupby('변경일자_년').agg({'차대번호':'count', '저감장치구분':[knd1, knd2]}).reset_index()
grp3.columns = ['연도', '말소대수', '말소저감장치(1종)', '말소저감장치(1종+SCR)']
grp3

,연도,말소대수,말소저감장치(1종),말소저감장치(1종+SCR)
0,2019,272934,8689,36
1,2020,428044,17014,87
2,2021,362632,21667,130
3,2022,126499,15687,76


In [348]:
# 연도별 조기폐차 대수
grp4 = dfe.groupby('말소일자_년').agg({'조기폐차최종승인YN':'count'}).reset_index()
grp4 = grp4.rename(columns={'말소일자_년':'연도', '조기폐차최종승인YN':'조기폐차'})
grp4

,연도,조기폐차
0,2005,1
1,2011,1
2,2013,1
3,2015,1
4,2018,1
5,2019,79951
6,2020,97488
7,2021,108406
8,2022,91716
9,2023,15957


In [349]:
# 4년간 차량 통계 기본 데이터셋
yr_list = []
for yr in range(2019, int(year) + 1):
    yr_list.append(str(yr))
base = pd.DataFrame({'연도':yr_list})
base

,연도
0,2019
1,2020
2,2021
3,2022


In [350]:
base1 = base.merge(grp1, on='연도', how='left')
base1

,연도,차량대수,저감장치(1종),저감장치(1종+SCR)
0,2019,NaN,NaN,NaN
1,2020,NaN,NaN,NaN
2,2021,NaN,NaN,NaN
3,2022,1162311.00,316374.00,2666.00


In [351]:
base2 = base1.merge(grp2, on='연도', how='left')
base2

,연도,차량대수,저감장치(1종),저감장치(1종+SCR),등록대수,등록저감장치(1종),등록저감장치(1종+SCR)
0,2019,NaN,NaN,NaN,68,9,1
1,2020,NaN,NaN,NaN,24,6,1
2,2021,NaN,NaN,NaN,9,2,0
3,2022,1162311.00,316374.00,2666.00,2,0,0


In [352]:
base3 = base2.merge(grp3, on='연도', how='left')
base3

,연도,차량대수,저감장치(1종),저감장치(1종+SCR),등록대수,등록저감장치(1종),등록저감장치(1종+SCR),말소대수,말소저감장치(1종),말소저감장치(1종+SCR)
0,2019,NaN,NaN,NaN,68,9,1,272934,8689,36
1,2020,NaN,NaN,NaN,24,6,1,428044,17014,87
2,2021,NaN,NaN,NaN,9,2,0,362632,21667,130
3,2022,1162311.00,316374.00,2666.00,2,0,0,126499,15687,76


In [353]:
base4 = base3.merge(grp4, on='연도', how='left')
base4

,연도,차량대수,저감장치(1종),저감장치(1종+SCR),등록대수,등록저감장치(1종),등록저감장치(1종+SCR),말소대수,말소저감장치(1종),말소저감장치(1종+SCR),조기폐차
0,2019,NaN,NaN,NaN,68,9,1,272934,8689,36,79951
1,2020,NaN,NaN,NaN,24,6,1,428044,17014,87,97488
2,2021,NaN,NaN,NaN,9,2,0,362632,21667,130,108406
3,2022,1162311.00,316374.00,2666.00,2,0,0,126499,15687,76,91716


In [354]:
base4.columns

Index(['연도', '차량대수', '저감장치(1종)', '저감장치(1종+SCR)', '등록대수', '등록저감장치(1종)',
       '등록저감장치(1종+SCR)', '말소대수', '말소저감장치(1종)', '말소저감장치(1종+SCR)', '조기폐차'],
      dtype='object')

In [355]:
base4[['차량대수', '조기폐차', '저감장치(1종)', '저감장치(1종+SCR)', '등록대수', '등록저감장치(1종)', '등록저감장치(1종+SCR)', '말소대수', '말소저감장치(1종)', '말소저감장치(1종+SCR)']].isnull().sum()

차량대수              3
조기폐차              0
저감장치(1종)          3
저감장치(1종+SCR)      3
등록대수              0
등록저감장치(1종)        0
등록저감장치(1종+SCR)    0
말소대수              0
말소저감장치(1종)        0
말소저감장치(1종+SCR)    0
dtype: int64

In [356]:
base4[['차량대수', '조기폐차', '저감장치(1종)', '저감장치(1종+SCR)', '등록대수', '등록저감장치(1종)', '등록저감장치(1종+SCR)', '말소대수', '말소저감장치(1종)', '말소저감장치(1종+SCR)']] = base4[['차량대수', '조기폐차', '저감장치(1종)', '저감장치(1종+SCR)', '등록대수', '등록저감장치(1종)', '등록저감장치(1종+SCR)', '말소대수', '말소저감장치(1종)', '말소저감장치(1종+SCR)']].fillna(0)

In [357]:
n = len(base4['연도'].unique())
for i in range(base4.shape[0] // n):
    for j in range(2, n+1):
        base4.loc[(i+1)*n - j, '차량대수'] = base4.loc[(i+1)*n - (j-1), '차량대수'] + base4.loc[(i+1)*n - (j-1), '말소대수'] - base4.loc[(i+1)*n - (j-1), '등록대수']
        base4.loc[(i+1)*n - j, '저감장치(1종)'] = base4.loc[(i+1)*n - (j-1), '저감장치(1종)'] + base4.loc[(i+1)*n - (j-1), '말소저감장치(1종)'] - base4.loc[(i+1)*n - (j-1), '등록저감장치(1종)']
        base4.loc[(i+1)*n - j, '저감장치(1종+SCR)'] = base4.loc[(i+1)*n - (j-1), '저감장치(1종+SCR)'] + base4.loc[(i+1)*n - (j-1), '말소저감장치(1종+SCR)'] - base4.loc[(i+1)*n - (j-1), '등록저감장치(1종+SCR)']

In [358]:
base4

,연도,차량대수,저감장치(1종),저감장치(1종+SCR),등록대수,등록저감장치(1종),등록저감장치(1종+SCR),말소대수,말소저감장치(1종),말소저감장치(1종+SCR),조기폐차
0,2019,2079451.00,370734.00,2958.00,68,9,1,272934,8689,36,79951
1,2020,1651431.00,353726.00,2872.00,24,6,1,428044,17014,87,97488
2,2021,1288808.00,332061.00,2742.00,9,2,0,362632,21667,130,108406
3,2022,1162311.00,316374.00,2666.00,2,0,0,126499,15687,76,91716


In [359]:
base5 = base4[['연도', '차량대수', '조기폐차', '저감장치(1종)', '저감장치(1종+SCR)']]
base5

,연도,차량대수,조기폐차,저감장치(1종),저감장치(1종+SCR)
0,2019,2079451.00,79951,370734.00,2958.00
1,2020,1651431.00,97488,353726.00,2872.00
2,2021,1288808.00,108406,332061.00,2742.00
3,2022,1162311.00,91716,316374.00,2666.00


In [360]:
base5['감소대수'] = base5['차량대수'].shift() - base5['차량대수']
base5

C:\Users\kbjung\AppData\Local\Temp\ipykernel_8792\2849714677.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base5['감소대수'] = base5['차량대수'].shift() - base5['차량대수']


,연도,차량대수,조기폐차,저감장치(1종),저감장치(1종+SCR),감소대수
0,2019,2079451.00,79951,370734.00,2958.00,NaN
1,2020,1651431.00,97488,353726.00,2872.00,428020.00
2,2021,1288808.00,108406,332061.00,2742.00,362623.00
3,2022,1162311.00,91716,316374.00,2666.00,126497.00


In [361]:
base5['자연감소'] = base5['감소대수'] - base5['조기폐차']
base5

C:\Users\kbjung\AppData\Local\Temp\ipykernel_8792\2601777811.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base5['자연감소'] = base5['감소대수'] - base5['조기폐차']


,연도,차량대수,조기폐차,저감장치(1종),저감장치(1종+SCR),감소대수,자연감소
0,2019,2079451.00,79951,370734.00,2958.00,NaN,NaN
1,2020,1651431.00,97488,353726.00,2872.00,428020.00,330532.00
2,2021,1288808.00,108406,332061.00,2742.00,362623.00,254217.00
3,2022,1162311.00,91716,316374.00,2666.00,126497.00,34781.00


In [362]:
base5['미장착'] = base5['차량대수'] - base5['저감장치(1종)'] - base5['저감장치(1종+SCR)']
base5

C:\Users\kbjung\AppData\Local\Temp\ipykernel_8792\3804496685.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base5['미장착'] = base5['차량대수'] - base5['저감장치(1종)'] - base5['저감장치(1종+SCR)']


,연도,차량대수,조기폐차,저감장치(1종),저감장치(1종+SCR),감소대수,자연감소,미장착
0,2019,2079451.00,79951,370734.00,2958.00,NaN,NaN,1705759.00
1,2020,1651431.00,97488,353726.00,2872.00,428020.00,330532.00,1294833.00
2,2021,1288808.00,108406,332061.00,2742.00,362623.00,254217.00,954005.00
3,2022,1162311.00,91716,316374.00,2666.00,126497.00,34781.00,843271.00


In [363]:
base5['테이블생성일자'] = today_date

C:\Users\kbjung\AppData\Local\Temp\ipykernel_8792\3755925745.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base5['테이블생성일자'] = today_date


In [364]:
base5.columns.to_list()

['연도',
 '차량대수',
 '조기폐차',
 '저감장치(1종)',
 '저감장치(1종+SCR)',
 '감소대수',
 '자연감소',
 '미장착',
 '테이블생성일자']

In [365]:
base5 = base5[[
    '연도',
    '차량대수',
    '자연감소',
    '조기폐차',
    '저감장치(1종)',
    '저감장치(1종+SCR)',
    '미장착',
    '테이블생성일자'
    ]]
base5

,연도,차량대수,자연감소,조기폐차,저감장치(1종),저감장치(1종+SCR),미장착,테이블생성일자
0,2019,2079451.00,NaN,79951,370734.00,2958.00,1705759.00,20230725
1,2020,1651431.00,330532.00,97488,353726.00,2872.00,1294833.00,20230725
2,2021,1288808.00,254217.00,108406,332061.00,2742.00,954005.00,20230725
3,2022,1162311.00,34781.00,91716,316374.00,2666.00,843271.00,20230725


In [366]:
cdict = {
    '연도':'YR', 
    '차량대수':'VHCL_MKCNT', 
    '자연감소':'NTRL_DCLN', 
    '조기폐차':'ELPDSRC', 
    '저감장치(1종)':'RDCDVC_1KND', 
    '저감장치(1종+SCR)':'RDCDVC_1KND_SCR', 
    '미장착':'UNMNTNG', 
    '테이블생성일자':'LOAD_DT', 
}
STD_BD_DAT_GRD5_REDUC_BIZ = base5.rename(columns=cdict)
STD_BD_DAT_GRD5_REDUC_BIZ.columns

Index(['YR', 'VHCL_MKCNT', 'NTRL_DCLN', 'ELPDSRC', 'RDCDVC_1KND',
       'RDCDVC_1KND_SCR', 'UNMNTNG', 'LOAD_DT'],
      dtype='object')

In [367]:
STD_BD_DAT_GRD5_REDUC_BIZ.columns.to_list()

['YR',
 'VHCL_MKCNT',
 'NTRL_DCLN',
 'ELPDSRC',
 'RDCDVC_1KND',
 'RDCDVC_1KND_SCR',
 'UNMNTNG',
 'LOAD_DT']

### [출력] STD_BD_DAT_GRD5_REDUC_BIZ

In [271]:
# expdf = STD_BD_DAT_GRD5_REDUC_BIZ
# table_nm = 'STD_BD_DAT_GRD5_REDUC_BIZ'.upper()

# # 테이블 생성
# sql = 'create table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     # if 'float' in expdf[column].dtype.name:
#     #     sql += column + ' float'
#     # elif 'int' in expdf[column].dtype.name:
#     #     sql += column + ' number'
#     # else:
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 5s
# we.import_from_pandas(expdf, table_nm)

In [368]:
# # 0s
# STD_BD_DAT_GRD5_REDUC_BIZ.to_csv(os.path.join(df3_fold, 'STD_BD_DAT_GRD5_REDUC_BIZ.csv'), index=False)

## 운행제한현황

In [275]:
lmt1.columns

Index(['차대번호', '법정동코드', '차량말소YN', '제원관리번호', '차량연식', '차종', '용도', '최초등록일자',
       '제작일자', '차량관리번호', '연료', '차종유형', 'DPF_YN', '시도코드', '시군구코드', '시도', '시군구',
       '조기폐차상태코드', '조기폐차최종승인YN', '말소일자', '계절제_1차', '계절제_2차', '계절제_3차',
       '계절제_4차', '비상시', '상시', '지역', '계절제_1차_일평균', '계절제_2차_일평균', '계절제_3차_일평균',
       '계절제_4차_일평균', '테이블생성일자'],
      dtype='object')

In [276]:
run_lmt1 = lmt1.copy()

In [278]:
run_lmt1.shape, len(run_lmt1['차대번호'].unique())

((1162311, 32), 1162311)

In [301]:
run_lmt1['DPF_YN'].value_counts(dropna=False)

DPF_YN
무    843271
유    319040
Name: count, dtype: int64

In [302]:
run_lmt1.loc[run_lmt1['DPF_YN'] == '무', '저감장치미장착'] = '미장착'
run_lmt1.head()

,차대번호,법정동코드,차량말소YN,제원관리번호,차량연식,차종,용도,최초등록일자,제작일자,차량관리번호,...,계절제_4차,비상시,상시,지역,계절제_1차_일평균,계절제_2차_일평균,계절제_3차_일평균,계절제_4차_일평균,테이블생성일자,저감장치미장착
0,KL3C3AHF27K000082,4481025000,N,A0310007202023307,2007,화물,개인용,20220711,20070627.00,KL3C3AHF27K000082-01,...,0.00,NaN,NaN,수도권외,0.00,0.00,0.00,0.00,20230724,미장착
1,KL3C3AHF27K000068,4372033032,N,A0310007202023307,2007,화물,개인용,20220413,20070618.00,KL3C3AHF27K000068-01,...,0.00,NaN,NaN,수도권외,0.00,0.00,0.00,0.00,20230724,미장착
2,1P3EW65G5XV503775,4157010600,N,10020204100011319,1999,승용,개인용,20211124,20190905,1P3EW65G5XV503775-01,...,0.00,NaN,NaN,수도권,0.00,0.00,0.00,0.00,20230724,미장착
3,1P3EW65G5YV606177,5011013700,N,10020204100001319,2000,승용,개인용,20211124,20190802,1P3EW65G5YV606177-01,...,0.00,NaN,NaN,수도권외,0.00,0.00,0.00,0.00,20230724,미장착
4,KMCYSS7JP7U084697,4423038028,N,A0810003403394206,2007,특수,개인용,20210826,20070227,KMCYSS7JP7U084697-01,...,0.00,NaN,NaN,수도권외,0.00,0.00,0.00,0.00,20230724,미장착


In [303]:
temp1 = run_lmt1.loc[run_lmt1['계절제_2차'] > 0, ['차대번호', '지역', '시도', '차종', '차종유형', '계절제_2차', '저감장치미장착']]
temp1.shape

(26288, 7)

In [304]:
temp2 = run_lmt1.loc[run_lmt1['계절제_2차'] > 1, ['차대번호', '지역', '시도', '차종', '차종유형', '계절제_2차']]
temp2.shape

(11576, 6)

In [306]:
grp1 = temp1.groupby(['지역', '시도', '차종', '차종유형']).agg({'차대번호':'count', '저감장치미장착':'count'}).reset_index()
grp1 = grp1.rename(columns={'차대번호':'적발차량대수', '저감장치미장착':'저공해미조치'})
grp1

,지역,시도,차종,차종유형,적발차량대수,저공해미조치
0,수도권,경기도,승용,대형,3039,1104
1,수도권,경기도,승용,소형,21,21
2,수도권,경기도,승용,중형,1899,711
3,수도권,경기도,승합,대형,25,12
4,수도권,경기도,승합,소형,3,3
...,...,...,...,...,...,...
158,수도권외,충청북도,특수,대형,9,7
159,수도권외,충청북도,특수,중형,1,1
160,수도권외,충청북도,화물,대형,213,78
161,수도권외,충청북도,화물,소형,200,126


In [307]:
grp2 = temp2.groupby(['지역', '시도', '차종', '차종유형'])['차대번호'].count().reset_index()
grp2 = grp2.rename(columns={'차대번호':'중복적발대수'})
grp2

,지역,시도,차종,차종유형,중복적발대수
0,수도권,경기도,승용,대형,1541
1,수도권,경기도,승용,소형,9
2,수도권,경기도,승용,중형,927
3,수도권,경기도,승합,대형,11
4,수도권,경기도,승합,중형,440
...,...,...,...,...,...
139,수도권외,충청북도,승합,중형,25
140,수도권외,충청북도,특수,대형,3
141,수도권외,충청북도,화물,대형,114
142,수도권외,충청북도,화물,소형,61


In [309]:
grp = grp1.merge(grp2, on=['지역', '시도', '차종', '차종유형'], how='left')
grp['계절관리제'] = '계절제_2차'
grp

,지역,시도,차종,차종유형,적발차량대수,저공해미조치,중복적발대수,계절관리제
0,수도권,경기도,승용,대형,3039,1104,1541.00,계절제_2차
1,수도권,경기도,승용,소형,21,21,9.00,계절제_2차
2,수도권,경기도,승용,중형,1899,711,927.00,계절제_2차
3,수도권,경기도,승합,대형,25,12,11.00,계절제_2차
4,수도권,경기도,승합,소형,3,3,NaN,계절제_2차
...,...,...,...,...,...,...,...,...
158,수도권외,충청북도,특수,대형,9,7,3.00,계절제_2차
159,수도권외,충청북도,특수,중형,1,1,NaN,계절제_2차
160,수도권외,충청북도,화물,대형,213,78,114.00,계절제_2차
161,수도권외,충청북도,화물,소형,200,126,61.00,계절제_2차


In [315]:
total_grp_lmt = pd.DataFrame()
for one in limit_season_rename_dict.values():
    temp1 = run_lmt1.loc[run_lmt1[one] > 0, ['차대번호', '지역', '시도', '차종', '차종유형', '저감장치미장착'] + [one]]
    temp2 = run_lmt1.loc[run_lmt1[one] > 1, ['차대번호', '지역', '시도', '차종', '차종유형'] + [one]]
    if temp1.shape[0] > 0 and temp2.shape[0] > 0:
        grp1 = temp1.groupby(['지역', '시도', '차종', '차종유형']).agg({'차대번호':'count', '저감장치미장착':'count'}).reset_index()
        grp1 = grp1.rename(columns={'차대번호':'적발차량대수', '저감장치미장착':'저공해미조치'})
        grp2 = temp2.groupby(['지역', '시도', '차종', '차종유형'])['차대번호'].count().reset_index()
        grp2 = grp2.rename(columns={'차대번호':'중복적발대수'})
        grp = grp1.merge(grp2, on=['지역', '시도', '차종', '차종유형'], how='left')
        grp['계절관리제'] = one
        total_grp_lmt = pd.concat([total_grp_lmt, grp], ignore_index=True)
    else:
        pass

In [316]:
total_grp_lmt

,지역,시도,차종,차종유형,적발차량대수,저공해미조치,중복적발대수,계절관리제
0,수도권,경기도,승용,대형,3039,1104,1541.00,계절제_2차
1,수도권,경기도,승용,소형,21,21,9.00,계절제_2차
2,수도권,경기도,승용,중형,1899,711,927.00,계절제_2차
3,수도권,경기도,승합,대형,25,12,11.00,계절제_2차
4,수도권,경기도,승합,소형,3,3,NaN,계절제_2차
...,...,...,...,...,...,...,...,...
493,수도권외,충청북도,특수,소형,1,1,NaN,계절제_4차
494,수도권외,충청북도,특수,중형,1,1,NaN,계절제_4차
495,수도권외,충청북도,화물,대형,25,24,9.00,계절제_4차
496,수도권외,충청북도,화물,소형,68,68,10.00,계절제_4차


In [330]:
total_grp_lmt['계절관리제'].unique()

array(['계절제_2차', '계절제_3차', '계절제_4차'], dtype=object)

In [325]:
total_grp_lmt['테이블생성일자'] = today_date

C:\Users\kbjung\AppData\Local\Temp\ipykernel_5296\3564358631.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_grp_lmt['테이블생성일자'] = today_date


In [326]:
total_grp_lmt.columns.to_list()

['계절관리제', '지역', '시도', '차종', '차종유형', '적발차량대수', '저공해미조치', '중복적발대수', '테이블생성일자']

In [320]:
total_grp_lmt = total_grp_lmt[[
    '계절관리제', 
    '지역', 
    '시도', 
    '차종', 
    '차종유형', 
    '적발차량대수', 
    '저공해미조치', 
    '중복적발대수', 
    '테이블생성일자', 
]]

In [327]:
cdict = {
    '계절관리제':'SEASON', 
    '지역':'RGN', 
    '시도':'CTPV', 
    '차종':'VHCTY_CD', 
    '차종유형':'VHCTY_TY', 
    '적발차량대수':'DSCL_VHCL_MKCNT', 
    '저공해미조치':'UNLEM', 
    '중복적발대수':'DUP_DSCL_MKCNT', 
    '테이블생성일자':'LOAD_DT', 
}
STD_BD_DAT_RUN_LMT_CURSTT = total_grp_lmt.rename(columns=cdict)
STD_BD_DAT_RUN_LMT_CURSTT.columns

Index(['SEASON', 'RGN', 'CTPV', 'VHCTY_CD', 'VHCTY_TY', 'DSCL_VHCL_MKCNT',
       'UNLEM', 'DUP_DSCL_MKCNT', 'LOAD_DT'],
      dtype='object')

In [328]:
STD_BD_DAT_RUN_LMT_CURSTT.columns.to_list()

['SEASON',
 'RGN',
 'CTPV',
 'VHCTY_CD',
 'VHCTY_TY',
 'DSCL_VHCL_MKCNT',
 'UNLEM',
 'DUP_DSCL_MKCNT',
 'LOAD_DT']

### [출력] STD_BD_DAT_RUN_LMT_CURSTT

In [ ]:
# expdf = STD_BD_DAT_RUN_LMT_CURSTT
# table_nm = 'STD_BD_DAT_RUN_LMT_CURSTT'.upper()

# # 테이블 생성
# sql = 'create table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     # if 'float' in expdf[column].dtype.name:
#     #     sql += column + ' float'
#     # elif 'int' in expdf[column].dtype.name:
#     #     sql += column + ' number'
#     # else:
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 5s
# we.import_from_pandas(expdf, table_nm)

In [329]:
# # 0s
# STD_BD_DAT_RUN_LMT_CURSTT.to_csv(os.path.join(df3_fold, 'STD_BD_DAT_RUN_LMT_CURSTT.csv'), index=False)

# code end